In [1]:
from functools import reduce  # python3 compatibility
from operator import mul
import json
import math
import os
import random
import statistics
import argparse
import concurrent.futures
from multiprocessing import Pool
import itertools
import math
import numpy as np
import time
from enum import Enum


In [2]:
def gen_tasks(task_num, max_capNum, capabilities):
    """
    Generate tasks, each task is represented by a list of capabilities it requires
    :param: `task_num`: the number of tasks
    :param: `max_capNum`: the maximum number of capabilities a task could require
    :param: `capabilities`: the list of capabilities
    :return: the list of tasks. Each task is represented by a list of capabilities it requires.
    """
    # n is the number of task, max_capNum is the maximum number of cap a task could require
    return [
        sorted(
            np.random.choice(
                a=capabilities, size=np.random.randint(3, max_capNum + 1), replace=False
            )
        )
        for j in range(0, task_num)
    ]


def gen_constraints(agent_num, task_num, power=1, a_min_edge=2, t_max_edge=5):
    """
    Generate agent's constraints, each agent is represented by a list of tasks it has capability to work on.
    :param: `agent_num`: the number of agents
    :param: `task_num`: the number of tasks
    :param: `power`: the power used to magnify the probability
    :param: `a_min_edge`: the minimum number of tasks an agent has the capabilities work on.
    :param: `t_max_edge`: the maximum number of agents that could work on a task.
    :return: For each agent, the list of tasks it has the capabilities to work on. For each task, the list of agents that could work on it.
    """

    # power is the inforce you put in the probabilities
    # the maximum tasks an agent could work on depends on the number of tasks available (e.g, if |T| = 1/2|A|, then roughly each agent can work on two tasks)

    # calculate the max and min edges for agents
    available_seats = math.floor(t_max_edge * task_num)
    a_taskInds = [[] for i in range(0, agent_num)]
    a_taskNums = []
    for i in range(0, agent_num):
        a_max_edge = min((available_seats - a_min_edge * (agent_num - 1 - i)), t_max_edge, task_num)
        a_min_edge = min(a_min_edge, a_max_edge)
        
        # radomly indicate the number of task the agent could work on, based on the maximum and minimum number of tasks the agent could work on
        a_taskNum = np.random.randint(a_min_edge, a_max_edge + 1)
        
        a_taskNums.append(a_taskNum)
        
        available_seats -= a_taskNum

    t_agents_counts = [0 for j in range(0, task_num)]  # each indicate the current number of agents on the task

    # make sure no further draw for those reached the maximum limit.
    t_indexes = [j for j in range(0, task_num) if t_agents_counts[j] < t_max_edge]

    for i, a_taskNum in enumerate(a_taskNums):
        if any(tc == 0 for tc in t_agents_counts):  # if there are tasks that have not been allocated to any agent
            t_prob = [
                (math.e ** (t_max_edge - t_agents_counts[j])) ** power
                for j in t_indexes
            ]  # power is used to manify the probability
            sum_prob = sum(t_prob)
            t_prop_2 = [prop / sum_prob for prop in t_prob]

            # draw tasks accounting to their current allocations
            a_taskInds[i] = list(
                np.random.choice(
                    a=t_indexes,
                    size=min(a_taskNum, len(t_indexes)),
                    replace=False,
                    p=[prop / sum_prob for prop in t_prob],
                )
            )
            # increase the chosen task counters
        else:
            a_taskInds[i] = list(
                np.random.choice(
                    a=t_indexes, size=min(a_taskNum, len(t_indexes)), replace=False
                )
            )

        for j in a_taskInds[i]:
            t_agents_counts[j] += 1

        # make sure no further draw for those reached the maximum limit.
        t_indexes = [
            j for j in range(0, task_num) if t_agents_counts[j] < t_max_edge
        ]

    # get also the list of agents for each task
    t_agents = [
        [i for i in range(0, agent_num) if j in a_taskInds[i]]
        for j in range(0, task_num)
    ]

    return a_taskInds, t_agents


def gen_agents(a_taskInds, tasks, max_capNum, capabilities, max_capVal):  
    # m is the number of task, max_capNum is the maximum number of cap a task could require, max_capVal is the maximum capability value
    """
    Generate agents, each agent is represented by a list of capabilities it has and a list of contribution values for each capability
    :param: `a_taskInds`: the list of list of tasks each agent could work on
    :param: `tasks`: the list of tasks, represented by a list of capabilities it requires
    :param: `max_capNum`: the maximum number of capabilities an agent could have
    :param: `capabilities`: the list of capabilities
    :param: `max_capVal`: the maximum value of a capability
    """
    caps_lists = []
    contri_lists = []
    for a_taskInd in a_taskInds:
        t_caps = [tasks[j] for j in a_taskInd]  # lists of caps that each task agent could perform

        caps_union = set(itertools.chain(*t_caps))  # union of unique caps of tasks that agent could perform.

        a_cap_num = np.random.randint(
            min(3, max_capNum, len(caps_union)), 
            min(len(caps_union), max_capNum) + 1
        )  # the num of caps the agent will have

        a_caps = set([np.random.choice(t_c) for t_c in t_caps])  # initial draw to guarantee the agent has some contribution to each of the task that the agent has the capability to perform.

        # Randomly draw the remaining capabilities, possibly none
        remaining_choices = list(caps_union.difference(a_caps))
        if remaining_choices != []:
            a_caps.update(
                np.random.choice(
                    remaining_choices,
                    min(max(0, a_cap_num - len(a_taskInd)), len(remaining_choices)),
                    replace=False,
                )
            )
        
        # a_caps.update(np.random.choice(remaining_choices, min(0,len(remaining_choices),a_cap_num-len(a_taskInd)),replace = False))

        caps_list = sorted(list(a_caps))
        contri_list = [
            (np.random.randint(1, max_capVal + 1) if c in caps_list else 0)
            for c in range(0, len(capabilities))
        ]

        caps_lists.append(caps_list)
        contri_lists.append(contri_list)

    return caps_lists, contri_lists



In [3]:
def task_reward(task, agents, gamma=1):
    # task is represented by a list of capabilities it requires, agents is a list agents, where each represented by a list cap contribution values
    """
    Calculate the reward of a single task
    :param: `task`: the list of capabilities the task requires
    :param: `agents`: the list of agents
    :param: `gamma`: the discount factor
    :return: the reward of the task
    """
    if agents == []:
        return 0
    else:
        return sum([max([agent[c] for agent in agents]) for c in task]) * (
            gamma ** len(agents)
        )


def sys_reward_agents(agents, tasks, allocation_structure, gamma=1):
    """
    Calculate the reward of the system, given the allocation structure: agent -> task
    """
    # allocation_structure is a vector of size M, each element indicate which task the agent is allocated to
    return sum(
        task_reward(task, [agent for i, agent in enumerate(agents) if allocation_structure[i] == j], gamma)
        for j, task in enumerate(tasks)
    )


def sys_rewards_tasks(tasks, agents, coalition_structure, gamma=1):
    """
    Calculate the reward of the system, given the coalition structure: task -> agents (coalition)
    """
    return sum(
        task_reward(task, [agents[i] for i in coalition_structure[j]], gamma)
        for j, task in enumerate(tasks)
    )





In [4]:

def agent_contribution(agents, tasks, query_agentIndex, query_taskIndex, coalition, constraints, gamma=1):
    """
    Return contribution of agent i to task j in coalition C_j
    
    = U_i(C_j, j) - U_i(C_j \ {i}, j) if i in C_j

    = U_i(C_j U {i}, j) - U_i(S, j) if i not in C_j
    """
    a_taskInds = constraints[0]
    if query_taskIndex == len(tasks):
        return 0
    if query_taskIndex not in a_taskInds[query_agentIndex]:
        return 0
    cur_reward = task_reward(tasks[query_taskIndex], [agents[i] for i in coalition], gamma)
    if query_agentIndex in coalition:
        return cur_reward - task_reward(tasks[query_taskIndex], [agents[i] for i in coalition if i != query_agentIndex], gamma)
    else:
        return task_reward(tasks[query_taskIndex], [agents[i] for i in coalition] + [agents[query_agentIndex]], gamma) - cur_reward



<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
C:\Users\An\AppData\Local\Temp\ipykernel_10732\1628021997.py:2: SyntaxWarning: invalid escape sequence '\ '
  """


In [5]:
def eGreedy2(
        agents : list[list[float]], 
        tasks : list[list[int]], 
        constraints : tuple[list[list[int]], list[list[int]]],
        coalition_structure : list[list[int]] = [],
        eps=0, 
        gamma=1
    ):
    re_assignment_count = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)
    allocation_structure = [task_num for i in range(0, agent_num)]  # each indicate the current task that agent i is allocated to, if = N, means not allocated
    if coalition_structure is None or coalition_structure == []:
        coalition_structure = [[] for j in range(0, task_num)] + [list(range(0, agent_num))]  # default coalition structure, the last one is dummy coalition
        cur_con = [0 for j in range(0, agent_num)]
    else:
        coalition_structure.append([])
        for j in range(0, task_num):
            for i in coalition_structure[j]:
                allocation_structure[i] = j
        cur_con = [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            for i, j in enumerate(allocation_structure)
        ]

    task_cons = [
        [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j in a_taskInds[i]
            else float("-inf")
            for j in range(0, task_num)
        ] + [0]
        for i in range(0, agent_num)
    ]
    # the last 0 indicate not allocated

    move_vals = [
        [
            task_cons[i][j] - cur_con[i] if j in a_taskInds[i] + [task_num] else -1000
            for j in range(0, task_num + 1)
        ]
        for i in range(0, agent_num)
    ]

    max_moveIndexs = [
        np.argmax([move_vals[i][j] for j in a_taskInds[i]] + [0])
        for i in range(0, agent_num)
    ]

    max_moveVals = [
        move_vals[i][a_taskInds[i][max_moveIndexs[i]]]
        if max_moveIndexs[i] < len(a_taskInds[i])
        else move_vals[i][task_num]
        for i in range(0, agent_num)
    ]

    iteration_count = 0
    while True:
        iteration_count += 1
        feasible_choices = [i for i in range(0, agent_num) if max_moveVals[i] > 0]
        if feasible_choices == []:
            break  # reach NE solution
        # when eps = 1, it's Random, when eps = 0, it's Greedy
        if np.random.uniform() <= eps:
            # exploration: random allocation
            a_index = np.random.choice(feasible_choices)
        else:
            # exploitation: allocationelse based on reputation or efficiency
            a_index = np.argmax(max_moveVals)
            
        t_index = a_taskInds[a_index][max_moveIndexs[a_index]] if max_moveIndexs[a_index] < len(a_taskInds[a_index]) else task_num

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != task_num:
            affected_a_indexes.extend(coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for i in coalition_structure[t_index]:
                task_cons[i][t_index] = agent_contribution(agents, tasks, i, t_index, coalition_structure[t_index], constraints, gamma)
                cur_con[i] = task_cons[i][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != task_num):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for i in coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        for i in affected_a_indexes:
            move_vals[i] = [
                task_cons[i][j] - cur_con[i]
                if j in a_taskInds[i] + [task_num]
                else -1000
                for j in range(0, task_num + 1)
            ]

        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for i in range(0, agent_num):
                if (i not in coalition_structure[t_ind]) and (t_ind in a_taskInds[i]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, coalition_structure[t_ind], constraints, gamma)
                    move_vals[i][t_ind] = task_cons[i][t_ind] - cur_con[i]

        max_moveIndexs = [
            np.argmax([move_vals[i][j] for j in a_taskInds[i] + [task_num]])
            for i in range(0, agent_num)
        ]
        max_moveVals = [
            move_vals[i][a_taskInds[i][max_moveIndexs[i]]]
            if max_moveIndexs[i] < len(a_taskInds[i])
            else move_vals[i][task_num]
            for i in range(0, agent_num)
        ]

    return (
        coalition_structure,
        sys_rewards_tasks(tasks, agents, coalition_structure, gamma),
        iteration_count,
        re_assignment_count,
    )


In [6]:
def random_solution_heterogeneous(agents, tasks, constraints, gamma=1):
    '''
    Randomly allocate tasks to agents
    '''
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = constraints[0]
    alloc = [np.random.choice(a_taskInds[i] + [task_num]) for i in range(0, agent_num)]
    return alloc, sys_reward_agents(agents, tasks, alloc, gamma)



In [7]:

def upperBound(capabilities, tasks, agents):
    """
    Calculate the upper bound of the system reward, where the system consists of tasks and agents with constraints.

    This mathematical upper bound is calculated by sorting the agents based on their contribution values for each capability, in descending order, then count `m`, the number of tasks that require each capability, and sum up the contribution values of the top `m` agents for each capability.
    
    :param: `capabilities`: the list of capabilities
    :param: `tasks`: the list of tasks
    :param: `agents`: the list of agents
    :return: the upper bound of the system reward
    """
    cap_ranked = [sorted([a[c] for a in agents], reverse=True) for c in capabilities] # Time complexity: O(len(capabilities) * log(len(capabilities)) * len(agents))
    cap_req_all = list(itertools.chain(*tasks)) # Time complexity: O(size of tasks capabilities combined), around O(len(tasks) * len(capabilities))
    cap_req_num = [cap_req_all.count(c) for c in capabilities] # Time complexity: O(len(cap_req_all) * len(capabilities)). However, can be optimized to O(len(cap_req_all)).
    return sum([sum(cap_ranked[c][:cap_req_num[c]]) for c in capabilities]) # Time complexity: O(len(cap_req_all))
    # Evaluated time complexity: max(O(len(capabilities) * log(len(capabilities)) * len(agents)), O(len(tasks) * len(capabilities)))


def upperBound_ver2(capabilities, tasks, agents, constraints):
    """
    Calculate the upper bound of the system reward, where the system consists of tasks and agents with constraints.

    This upper bound is calculated by sorting the agents based on their contribution values for each capability, in descending order, then iteratively allocate the top agents to the tasks that require that capability.

    This allows for a more precise upper bound than upperBound, since it takes into account the `constraints`: the top agents might only be able to work on the same limited tasks.

    :param: `capabilities`: the list of capabilities
    :param: `tasks`: the list of tasks
    :param: `agents`: the list of agents
    :param: `constraints`: the list of constraints
    :return: the upper bound of the system reward
    """
    agent_num = len(agents)
    task_num = len(tasks)
    a_taskInds = constraints[0]
    cap_req_all = list(itertools.chain(*tasks))
    cap_req_num = [cap_req_all.count(c) for c in capabilities]

    sys_rewards = 0
    for c in capabilities:
        
        a_cap_vals = [agent[c] for agent in agents]

        # the list of tasks that each agent has the capability to perform and that require the capability c
        a_cap_tasks = [[j for j in a_taskInd if j != task_num and c in tasks[j]] for a_taskInd in a_taskInds] 

        # sort the agents based on their contribution values for the capability c, in descending order
        cap_rank_pos = np.argsort(a_cap_vals)[::-1]

        a_cap_vals_ordered = [0 for _ in range(0, agent_num)]
        a_cap_tasks_ordered = [[] for _ in range(0, agent_num)]
        for p, pos in enumerate(cap_rank_pos):
            a_cap_vals_ordered[p] = a_cap_vals[pos]
            a_cap_tasks_ordered[p] = a_cap_tasks[pos]

        cap_rewards = a_cap_vals_ordered[0]
        cap_tasks = set(a_cap_tasks_ordered[0])
        a_cap_num = 1
        for a_iter in range(1, agent_num):
            cap_tasks = cap_tasks.union(set(a_cap_tasks_ordered[a_iter]))
            if len(cap_tasks) > a_cap_num:
                cap_rewards += a_cap_vals_ordered[a_iter]
                a_cap_num += 1
            # break if they got enough agents to contribute the number of required cap c
            if (a_cap_num >= cap_req_num[c]):  
                break
        sys_rewards += cap_rewards
    return sys_rewards



In [8]:
class NodeType(Enum):
    AND = "AND"
    OR = "OR"
    LEAF = "LEAF"
    DUMMY = "DUMMY"

def reverse_node_type(node_type):
    if node_type == NodeType.AND:
        return NodeType.OR
    elif node_type == NodeType.OR:
        return NodeType.AND
    else:
        return node_type





def gen_tree(
        num_leaves: int, 
        min_num_internals : int = 1,  
        max_num_internals : int = None, 
        min_depth : int = 1, 
        max_depth : int = None, 
        min_degree : int = 2, 
        max_degree : int = None,
        min_leaf_depth: int = 0,
    ):
    """
    Generates a random tree with the given number of leafs, minimum depth, maximum depth, maximum and minimum depth (number of children possible per internal (non-leaf) node), and maximum and minimum number of internal nodes.

    If you need exact number of internal nodes, set min_num_internals = max_num_internals = exact number of internal nodes.

    If you need exact depth, set min_depth = max_depth = exact depth.

    If you need exact degree, set min_degree = max_degree = exact degree.
    """

    if max_degree is None:
        max_degree = num_leaves

    if max_num_internals is None:
        max_num_internals = num_leaves - 1

    if max_depth is None:
        max_depth = float("inf")

    depth_info : dict[int, int] = {}
    parent_info : dict[int, int] = {}
    children_info : dict[int, list[int]] = {}
    
    global_id_iterator = 0
    root_id = global_id_iterator
    children_info[root_id] = []
    depth_info[root_id] = 0
    
    leaves_by_depth : list[list[int]] = [[root_id]] # depth -> list of leaves of that depth

    current_num_leaves = 1
    current_num_internals = 0

    while current_num_leaves < num_leaves:

        shallow_leaves = {d: l for d, l in enumerate(leaves_by_depth) if len(l) > 0 and d < min_leaf_depth}
        if len(shallow_leaves) > 0:
            chosen_depth = random.choice(list(shallow_leaves.keys()))

            # Choose a random value for the degree of the parent
            
            current_num_internals += 1
            
            min_tba_num_leaves = sum([len(l) * (min_degree ** (min_leaf_depth - d) - 1) for d, l in shallow_leaves.items()])

            tba_degree_upper_bound_2 = min_degree + (num_leaves - current_num_leaves - min_tba_num_leaves) // (min_degree ** (min_leaf_depth - chosen_depth - 1))

            current_num_leaves -= 1

            tba_degree_lower_bound = max(min_degree, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))

            tba_degree_upper_bound_1 = (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals)

            tba_degree_upper_bound = min(max_degree, tba_degree_upper_bound_1, tba_degree_upper_bound_2)
            
            if tba_degree_lower_bound > tba_degree_upper_bound:
                raise Exception("No valid tree exists with the given parameters.")
            
            tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)

            parent_id_index = random.randint(0, len(leaves_by_depth[chosen_depth]) - 1)
            parent_id = leaves_by_depth[chosen_depth].pop(parent_id_index)

        elif len(leaves_by_depth) - 1 < min_depth:
            chosen_depth = len(leaves_by_depth) - 1
            parent_id_index = random.randint(0, len(leaves_by_depth[chosen_depth]) - 1)
            parent_id = leaves_by_depth[chosen_depth].pop(parent_id_index)

            current_num_internals += 1
            current_num_leaves -= 1
            
            # Choose a random value for the degree of the parent
            
            tba_degree_lower_bound = max(min_degree, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))
            
            tba_degree_upper_bound = min(max_degree, (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals))
            
            tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)

        else:
            # Choose a random depth for the parent (the leaf to add children to)
            depths_options = [d for d, v in enumerate(leaves_by_depth) if len(v) > 0 and d < max_depth]

            if len(depths_options) > 0:
                
                # Choose a random depth for the parent (the leaf to add children to)
                chosen_depth = random.choice(depths_options)
                
                # Choose a random leaf from the current depth
                parent_id_index = random.randint(0, len(leaves_by_depth[chosen_depth]) - 1)
                parent_id = leaves_by_depth[chosen_depth].pop(parent_id_index)

                current_num_internals += 1
                current_num_leaves -= 1
                
                # Choose a random value for the degree of the parent
                
                tba_degree_lower_bound = max(min_degree, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))
                
                tba_degree_upper_bound = min(max_degree, (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals))
                
                tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)

            else:
                # Choose a random internal non-full node to add children to.
                non_max_degree_node_options = [_node for _node, _children in children_info.items() if len(_children) < max_degree]
                
                if len(non_max_degree_node_options) == 0:
                    raise Exception("No valid tree exists with the given parameters.")
                    
                # Select a pair of nodes, where the first one is not maximum degree and the second one is not minimum degree, such that the first one is higher (closer to the root) than the second one

                highest_node = min(non_max_degree_node_options, key=lambda _node: depth_info[_node], default=None)

                non_min_degree_node_options = [_node for _node, _children in children_info.items() if len(_children) > min_degree]
                
                deepest_node = max(non_min_degree_node_options, key=lambda _node: depth_info[_node], default=None)

                # If no such pair exists, then the method is not valid. We move to another method: add leafs to existing internal nodes.
                if highest_node is not None and deepest_node is not None and depth_info[highest_node] < depth_info[deepest_node]:
                
                    # Move an edge under the deepest_non_min_degree_node to under the highest_non_max_degree_node
                    
                    # Choose a random child of the deepest_non_min_degree_node
                    child_id = random.choice(children_info[deepest_node])
                    # Remove the child from the children list of the deepest_non_min_degree_node
                    children_info[deepest_node].remove(child_id)
                    # Add the child to the children list of the highest_non_max_degree_node
                    children_info[highest_node].append(child_id)
                    # Update the parent_info
                    parent_info[child_id] = highest_node
                    # Update the depth_info
                    depth_info[child_id] = depth_info[highest_node] + 1
                    # Update depth_info of all descendants of the child, using BFS
                    queue = [child_id]
                    while len(queue) > 0:
                        current_node = queue.pop(0)
                        old_depth = depth_info[current_node]
                        new_depth = depth_info[parent_info[current_node]] + 1
                        if current_node not in children_info:
                            # Update leaves
                            leaves_by_depth[old_depth].remove(current_node)
                            leaves_by_depth[new_depth].append(current_node)
                        depth_info[current_node] = new_depth
                        queue += children_info[current_node]
                    continue
                
                # else:
                
                # Add leafs to existing internal nodes.

                parent_id = random.choice(non_max_degree_node_options)

                # Get the depth of the parent
                chosen_depth = depth_info[parent_id]

                # Choose a random value for the to-be-added degree of the parent

                tba_degree_lower_bound = max(1, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))
                
                tba_degree_upper_bound = min(
                    max_degree - len(children_info[parent_id]),
                    (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals)
                )

                if tba_degree_lower_bound > tba_degree_upper_bound:
                    raise Exception("No valid tree exists with the given parameters.")
                
                tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)
            
        if parent_id not in children_info:
            children_info[parent_id] = []
        
        if chosen_depth + 1 >= len(leaves_by_depth):
            leaves_by_depth.append([])

        for _ in range(tba_degree):
            global_id_iterator += 1
            leaves_by_depth[chosen_depth + 1].append(global_id_iterator)
            parent_info[global_id_iterator] = parent_id
            depth_info[global_id_iterator] = chosen_depth + 1
            children_info[parent_id].append(global_id_iterator)
            current_num_leaves += 1

    return depth_info, parent_info, children_info, leaves_by_depth



def assign_node_type(
        depth_info: dict[int, int], 
        children_info: dict[int, int], 
        leaf_nodes : list[int], 
        root_node_id : int = 0, 
        strict_and_or: bool = True, 
        root_node_type : NodeType = None
    ):
    """
    Randomly assigns a node type to each node in the tree, where the node type is either "AND", "OR", or "LEAF".
    """
    node_type_info = {}
    if root_node_type is None:
        root_node_type = random.choice([NodeType.AND, NodeType.OR])
    reversed_root_node_type = reverse_node_type(root_node_type)
    node_type_info[root_node_id] = root_node_type
    for node_id in leaf_nodes:
        node_type_info[node_id] = NodeType.LEAF
    for node_id, node_children_ids in children_info.items():
        if node_children_ids is None or len(node_children_ids) == 0:
            node_type_info[node_id] = NodeType.LEAF
            continue

        if node_id == root_node_id:
            continue
        
        if strict_and_or:
            node_depth = depth_info[node_id]
            node_type_info[node_id] = root_node_type if node_depth % 2 == 0 else reversed_root_node_type
        else:
            node_type_info[node_id] = random.choice([NodeType.AND, NodeType.OR])

    return node_type_info


def gen_tree_simple(
        min_depth : int = 1, 
        max_depth : int = 1000, 
        min_degree : int = 2, 
        max_degree : int = 1000,
        min_leaf_depth: int = 0,
        eps: float = 0.1,
    ):
    """
    Generates a random tree with the given minimum depth, maximum depth, maximum and minimum depth (number of children possible per internal (non-leaf) node)

    If you need exact depth, set min_depth = max_depth = exact depth.

    If you need exact degree, set min_degree = max_degree = exact degree.

    eps is the probability of continuing the tree generation process after the minimum depth and minimum leaf depth conditions are satisfied.
    """

    depth_info : dict[int, int] = {}
    parent_info : dict[int, int] = {}
    children_info : dict[int, list[int]] = {}
    
    global_id_iterator = 0
    root_id = global_id_iterator
    children_info[root_id] = []
    depth_info[root_id] = 0
    
    leaves_by_depth : list[list[int]] = [[root_id]] # depth -> list of leaves of that depth

    must_continue = True


    while must_continue:

        shallow_leaves = {d: l for d, l in enumerate(leaves_by_depth) if len(l) > 0 and d < min_leaf_depth}
        
        if len(shallow_leaves) > 0:
            chosen_depth = random.choice(list(shallow_leaves.keys()))

        elif len(leaves_by_depth) - 1 < min_depth:
            chosen_depth = len(leaves_by_depth) - 1

        else:
            # Choose a random depth for the parent (the leaf to add children to)
            depths_options = [d for d, v in enumerate(leaves_by_depth) if len(v) > 0 and d < max_depth]

            if len(depths_options) > 0:
                
                # Choose a random depth for the parent (the leaf to add children to)
                chosen_depth = random.choice(depths_options)

                must_continue = random.random() < eps

            else:
                break

        # Choose a random leaf from the current depth as the parent
        parent_id_index = random.randint(0, len(leaves_by_depth[chosen_depth]) - 1)
        parent_id = leaves_by_depth[chosen_depth].pop(parent_id_index)
        
        # Choose a random value for the to-be-added degree of the parent
        tba_degree = random.randint(min_degree, max_degree)

        if parent_id not in children_info:
            children_info[parent_id] = []
        
        if chosen_depth + 1 >= len(leaves_by_depth):
            leaves_by_depth.append([])

        for _ in range(tba_degree):
            global_id_iterator += 1
            leaves_by_depth[chosen_depth + 1].append(global_id_iterator)
            parent_info[global_id_iterator] = parent_id
            depth_info[global_id_iterator] = chosen_depth + 1
            children_info[parent_id].append(global_id_iterator)

    return depth_info, parent_info, children_info, leaves_by_depth




In [9]:

def get_leaves_list_info(parent_info : dict[int, int], leaf_nodes : list[int]):
    """
    For each node in the tree, get the list of leaves that are descendants of that node.

    For each leaf node, get the path from that leaf node to the root node.
    """
    leaves_list_info : dict[int, list[int]] = { }
    for leaf_id in leaf_nodes:
        # leaves_list_info[leaf_id].append(leaf_id)
        current_node_id = leaf_id
        leaves_list_info[leaf_id] = [leaf_id]
        while current_node_id in parent_info:
            parent_id = parent_info[current_node_id]
            if parent_id not in leaves_list_info:
                leaves_list_info[parent_id] = [leaf_id]
            else:
                leaves_list_info[parent_id].append(leaf_id)
            current_node_id = parent_id
    
    return leaves_list_info




def get_nodes_constraints(node_type_info : dict[int, int], leaves_list_info : dict[int, list[int]], leaf2task: dict[int, int], constraints):
    """
    For each node in the tree, get the list of agents that can perform under that node, and the list of nodes that each agent can perform on.
    """
    nodes_agents_info = { node : [] for node in node_type_info }
    for node in node_type_info:
        if node_type_info[node] == NodeType.LEAF:
            nodes_agents_info[node] = constraints[1][leaf2task[node]]
        else:
            nodes_agents_info[node] = list(set(itertools.chain(
                *[constraints[1][leaf2task[leaf]] for leaf in leaves_list_info[node]]
            ))) # Concat lists and remove duplicates

    a_nodes = [[] for a in constraints[0]]
    for n_id, n_agents in nodes_agents_info.items():
        for a in n_agents:
            a_nodes[a].append(n_id)

    return a_nodes, nodes_agents_info



In [10]:


def traverse_tree(children_info : dict[int, list[int]], order='dfs', root_node_id=0):
    """
    Traverse tree using depth-first search or breath-first search.
    
    Returns a generator that yields the node ids.
    """
    frontier_pop_index = 0 if order.lower() == 'bfs' else -1
    frontier = [root_node_id]
    while len(frontier) > 0:
        node_id = frontier.pop(frontier_pop_index)
        yield node_id
        if node_id in children_info:
            for child_id in children_info[node_id]:
                frontier.append(child_id)



In [11]:

def get_cap_vector(capabilities : list[int], tasks : list[list[int]], query_taskId : int):
    """
    Get the capability requirement vector of a task.
    
    Also called a "ubc vector".
    """
    cap_vec = np.zeros(len(capabilities))
    for c in tasks[query_taskId]:
        cap_vec[c] = 1
    return cap_vec


def get_cap_vector_all(capabilities : list[int], tasks : list[list[int]]):
    """
    Get the capability requirement vectors of all tasks.

    Also called a "ubc vector".
    """
    return [get_cap_vector(capabilities, tasks, j) for j in range(0, len(tasks))]


def calculate_ubc_vectors(
        node_type_info : dict[int, NodeType],
        parent_info : dict[int, int], 
        leaves_list_info : dict[int, list[int]],
        leaf2task : dict[int, int],
        tasks_capVecs : list[np.ndarray],
        capabilities : list[int], 
        query_nodeId : int
    ):
    """
    For each descendant of 'query_nodeId', generate a "ubc vector", a vector of values.

    Each value in the vector represents each capability and the maximum number of agents' capacity values needed to perform that capability under that node (i.e. subtree, i.e. branch). "ubc" stands for "upper bound capability".
    """
    if node_type_info[query_nodeId] == NodeType.LEAF:
        return {query_nodeId : tasks_capVecs[leaf2task[query_nodeId]]}
    
    leaf_nodes = leaves_list_info[query_nodeId] if query_nodeId in leaves_list_info else []
    
    ubcv_info = { n_id : np.zeros(len(capabilities)) for n_id in leaf_nodes }
    
    for leaf_id in leaf_nodes:
        ubcv_info[leaf_id] = tasks_capVecs[leaf2task[leaf_id]]
        current_node_id = leaf_id
        while current_node_id in parent_info and current_node_id != query_nodeId:
            prev_node_id = current_node_id
            current_node_id = parent_info[current_node_id]
            node_type = node_type_info[current_node_id]
            if node_type == NodeType.OR:
                ubcv_info[current_node_id] = np.max([ubcv_info.get(current_node_id, np.zeros(len(capabilities))), ubcv_info[prev_node_id]], axis=0)
            elif node_type == NodeType.AND:
                ubcv_info[current_node_id] = np.sum([ubcv_info.get(current_node_id, np.zeros(len(capabilities))), ubcv_info[prev_node_id]], axis=0)

    return ubcv_info
    

def upperbound_node(
        ubcv_info : dict[int, np.ndarray],
        capabilities : list[int], 
        agents : list[list[float]],
        nodes_constraints : tuple[list[list[int]], dict[int, list[int]]],
        query_nodeId=0 
    ):
    """
    Calculate the upper bound of the system reward, i.e. at the root of the AND-OR goal tree.
    """
    nodes_agents = nodes_constraints[1]

    caps_ranked = [sorted([agents[i][c] for i in nodes_agents[query_nodeId]], reverse=True) for c in capabilities]

    cap_req_num = ubcv_info[query_nodeId]
    
    return sum([sum(caps_ranked[c][:int(cap_req_num[c])]) for c in capabilities])


def upperbound_node_all(
        children_info : dict[int, list[int]],
        ubcv_info : dict[int, np.ndarray],
        capabilities : list[int], 
        agents : list[list[float]],
        nodes_constraints : tuple[list[list[int]], dict[int, list[int]]],
        query_nodeId=0 
    ):
    """
    Calculate the upper bound of the reward (utility) at each descendant of the queried node in the AND-OR goal tree.
    """

    descendant_nodes = list(traverse_tree(children_info, root_node_id=query_nodeId))
    
    return { 
        node_id : upperbound_node(
            ubcv_info,
            capabilities,
            agents,
            nodes_constraints,
            query_nodeId=node_id
        )
        for node_id in descendant_nodes
    }



def upperbound_node_all_min(
        nodes_upper_bound : dict[int, float],
        node_type_info : dict[int, NodeType],
        children_info : dict[int, list[int]],
        query_nodeId=0
    ):
    """
    Calculate the upper bound of the reward (utility) at each node of the AND-OR goal tree.

    Refine the upper bound by taking the minimum of the upper bound calculated from the children nodes, and the upper bound calculated from the current node.
    """

    nodes_upper_bound_min = { node_id : 0 for node_id in nodes_upper_bound }

    def _min_upper_bound(node_id : int):
        node_type = node_type_info[node_id]

        if node_type == NodeType.LEAF:
            nodes_upper_bound_min[node_id] = nodes_upper_bound[node_id]

        elif node_type == NodeType.OR:
            nodes_upper_bound_min[node_id] = max(_min_upper_bound(child_id) for child_id in children_info[node_id])

        elif node_type == NodeType.AND:
            nodes_upper_bound_min[node_id] = sum(_min_upper_bound(child_id) for child_id in children_info[node_id])

        else:
            raise Exception("Unsupported node type")

        nodes_upper_bound_min[node_id] = min(nodes_upper_bound[node_id], nodes_upper_bound_min[node_id])
        return nodes_upper_bound_min[node_id]
        
    _min_upper_bound(query_nodeId)

    return nodes_upper_bound_min



In [12]:

def sys_reward_agents(agents, tasks, allocation_structure, gamma=1):
    """
    Calculate the reward of the system, given the allocation structure: agent -> task
    """
    # allocation_structure is a vector of size M, each element indicate which task the agent is allocated to
    return sum(
        task_reward(task, [agent for i, agent in enumerate(agents) if allocation_structure[i] == j], gamma)
        for j, task in enumerate(tasks)
    )


def sys_rewards_tasks(tasks, agents, coalition_structure, gamma=1):
    """
    Calculate the reward of the system, given the coalition structure: task -> agents (coalition)
    """
    return sum(
        task_reward(task, [agents[i] for i in coalition_structure[j]], gamma)
        for j, task in enumerate(tasks)
    )


In [13]:

def sys_rewards_tree_agents(
        node_type_info : dict[int, NodeType],
        children_info : dict[int, list[int]],
        leaf2task : dict[int, int], 
        tasks : list[list[int]], 
        agents : list[list[int]], 
        allocation_structure : list[int], 
        root_node_id=0, 
        gamma=1
    ):
    """
    Calculate the reward of the system, given the allocation structure: agent -> task
    """
    def sys_rewards_node(node_id : int):
        
        node_type = node_type_info[node_id]
        
        if node_type == NodeType.LEAF or node_type == NodeType.DUMMY:
            return task_reward(tasks[leaf2task[node_id]], [agent for i, agent in enumerate(agents) if allocation_structure[i] == node_id], gamma)
        
        child_rewards = [sys_rewards_node(child_id) for child_id in children_info[node_id]]
        
        if node_type == NodeType.AND:
            return sum(child_rewards)
        elif node_type == NodeType.OR:
            return max(child_rewards)
        
    return sys_rewards_node(root_node_id)


def sys_rewards_tree_tasks(
        node_type_info : dict[int, NodeType],
        children_info : dict[int, list[int]],
        leaf2task : dict[int, int], 
        tasks : list[list[int]], 
        agents : list[list[int]], 
        coalition_structure : list[list[int]], 
        root_node_id=0, 
        gamma=1
    ):
    """
    Calculate the reward of the system, given the coalition structure: task -> agents (coalition)
    """
    def sys_rewards_node(node_id : int):
        
        node_type = node_type_info[node_id]
        
        if node_type == NodeType.LEAF or node_type == NodeType.DUMMY:
            task_id = leaf2task[node_id]
            return task_reward(tasks[task_id], [agents[i] for i in coalition_structure[task_id]], gamma)
        
        child_rewards = [sys_rewards_node(child_id) for child_id in children_info[node_id]]
        
        if node_type == NodeType.AND:
            return sum(child_rewards)
        elif node_type == NodeType.OR:
            return max(child_rewards)
        
    return sys_rewards_node(root_node_id)


In [14]:

def random_solution_and_or_tree(
        node_type_info : dict[int, NodeType],
        children_info : dict[int, list[int]],
        leaf2task : dict[int, int], 
        tasks : list[list[int]], 
        agents : list[list[int]],
        constraints, 
        gamma=1
    ):
    '''
    Randomly allocate tasks to agents
    '''
    dummy_task_id = len(tasks)
    agent_num = len(agents)
    a_taskInds = constraints[0]
    allocation_structure = [np.random.choice(a_taskInds[i] + [dummy_task_id]) for i in range(0, agent_num)]
    return allocation_structure, sys_rewards_tree_agents(node_type_info, children_info, leaf2task, tasks, agents, allocation_structure, gamma=gamma)

In [15]:

def get_node_value_info(
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]], 
        leaf2task: list[int],
        coalition_structure: list[list[int]],
        root_node_id=0,
    ):
    """
    Calculate the value of each node in the tree, given the allocation structure.
    """
    node_value_info = {}
    def get_node_value(node_id: int):
        
        if node_id in node_value_info:
            return node_value_info[node_id]
        
        node_type = node_type_info[node_id]
        
        if node_type == NodeType.LEAF:
            task_id = leaf2task[node_id]
            node_value_info[node_id] = task_reward(task_id, coalition_structure[task_id])
        
        elif node_type == NodeType.AND:
            node_value_info[node_id] = sum(get_node_value(child_id) for child_id in children_info[node_id])
        
        else: # OR node
            node_value_info[node_id] = max(get_node_value(child_id) for child_id in children_info[node_id])

        return node_value_info[node_id]
    
    get_node_value(root_node_id)

    return node_value_info


def get_cur_con_tree(
        query_aId: int,
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]], 
        parent_info: dict[int, int],
        task2leaf: list[int],
        allocation_structure: list[int],
        node_value_info: dict[int, float],
        cur_con_info: list[float],
        root_node_id=0,
    ):
    """
    Calculate the change in nodes values when an agent defects from its current task.
    """
    nodes_alt_value = {}

    task_id = allocation_structure[query_aId]

    if task_id == len(task2leaf):
        return nodes_alt_value, 0
    
    current_node = task2leaf[allocation_structure[query_aId]]
    
    value_lost = cur_con_info[query_aId]
    
    nodes_alt_value[current_node] = node_value_info[current_node] - value_lost
    
    while current_node != root_node_id and value_lost > 0:
        parent_node = parent_info[current_node]
        if node_type_info[parent_node] == NodeType.AND:
            nodes_alt_value[parent_node] = node_value_info[parent_node] - value_lost
        else: # OR node
            new_parent_value = max(nodes_alt_value.get(child_id, node_value_info[child_id]) for child_id in children_info[parent_node])
            value_lost = node_value_info[parent_node] - new_parent_value
            if value_lost > 0:
                nodes_alt_value[parent_node] = new_parent_value
            else:
                break
        current_node = parent_node
    
    return nodes_alt_value, value_lost


def get_move_val_tree(
        query_aId: int,
        query_tId: int,
        deflect_nodes_alt_value: dict[int, float],
        node_type_info: dict[int, NodeType],
        parent_info: dict[int, int],
        task2leaf: list[int],
        node_value_info: dict[int, float],
        task_cons_info: list[list[float]],
        root_node_id=0,
        value_added_benchmark=0,
    ):

    nodes_alt_value = {}
    
    value_added = task_cons_info[query_aId][query_tId]

    current_node = task2leaf[query_tId]
    
    nodes_alt_value[current_node] = deflect_nodes_alt_value.get(current_node, node_value_info[current_node]) + value_added
    
    while current_node != root_node_id and value_added > 0 and value_added >= value_added_benchmark:
        parent_node = parent_info[current_node]
        current_parent_value = deflect_nodes_alt_value.get(parent_node, node_value_info[parent_node])
        if node_type_info[parent_node] == NodeType.AND:
            nodes_alt_value[parent_node] = current_parent_value + value_added
        else: # OR node
            if nodes_alt_value[current_node] > current_parent_value:
                value_added = nodes_alt_value[current_node] - current_parent_value
                nodes_alt_value[parent_node] = nodes_alt_value[current_node]
            else:
                value_added = 0
                break
        current_node = parent_node
    
    return nodes_alt_value, value_added



In [16]:

def treeGNE(
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]], 
        parent_info: dict[int, int],
        task2leaf: list[int],
        leaf2task: dict[int, int],
        tasks: list[list[int]],
        agents: list[dict[int, float]],
        constraints,
        coalition_structure : list[list[int]] = [],
        selected_tasks : list[int] = None,
        eps=0, 
        gamma=1,
        root_node_id=0,
    ):
    
    """
    GreedyNE on an AND-OR tree.

    At each iteration, calculate the change in nodes values when an agent defects from its current task, and the change in nodes values when an agent moves to a new task.
    """
    re_assignment_count = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)

    if selected_tasks is None:
        selected_tasks = list(range(len(tasks)))
        task_selected = [True for i in range(len(tasks))]

    else:    
        task_selected = [False for i in range(len(tasks))]
        for j in selected_tasks:
            task_selected[j] = True

    allocation_structure = [task_num for i in range(0, agent_num)]  # each indicate the current task that agent i is allocated to, if = N, means not allocated
    if coalition_structure is None or coalition_structure == []:
        coalition_structure = [[] for j in range(0, task_num)] + [list(range(0, agent_num))]  # default coalition structure, the last one is dummy coalition
        cur_con = [0 for j in range(0, agent_num)]
    else:

        if len(coalition_structure) < task_num:
            coalition_structure.append([])

        for j in range(0, task_num):
            if not task_selected[j]:
                coalition_structure[len(task_num)] += coalition_structure[j]
                coalition_structure[j] = []

        for j in range(0, task_num):
            for n_id in coalition_structure[j]:
                allocation_structure[n_id] = j

        cur_con = [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j != task_num and task_selected[j]
            else 0
            for i, j in enumerate(allocation_structure)
        ]

    task_cons = [
        [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j in a_taskInds[i] and task_selected[j]
            else float("-inf")
            for j in range(0, task_num)
        ] + [0]
        for i in range(0, agent_num)
    ]
    # the last 0 indicate not allocated


    node_value_info = get_node_value_info(node_type_info, children_info, leaf2task, coalition_structure, root_node_id)

    info_get_cur_con = {
        i: get_cur_con_tree(i, node_type_info, children_info, parent_info, task2leaf, allocation_structure, node_value_info, cur_con, root_node_id)
        for i in range(agent_num)
    }

    deflect_nodes_alt_value_info = {
        i: info_get_cur_con[i][0]
        for i in range(agent_num)
    }

    value_lost_info = {
        i: info_get_cur_con[i][1]
        for i in range(agent_num)
    }

    info_get_move_val_tree = {
        i: {
            j: get_move_val_tree(i, j, deflect_nodes_alt_value_info[i], node_type_info, parent_info, task2leaf, node_value_info, task_cons, root_node_id)
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }

    added_nodes_alt_value_info = {
        i: {
            j: info_get_move_val_tree[i][j][0]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }

    value_added_info = {
        i: {
            j: info_get_move_val_tree[i][j][1]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }

    sys_improvement_values_info = {
        i: {
            j: value_added_info[i][j] - value_lost_info[i]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }

    max_sys_improvement_values = {
        i: max(sys_improvement_values_info[i].items(), key=lambda x: x[1])
        for i in range(agent_num)
    }


    iteration_count = 0
    while True:
        iteration_count += 1
        feasible_choices = [i for i in range(0, agent_num) if max_sys_improvement_values[i][1] > 0]
        if feasible_choices == []:
            break  # reach NE solution
        # when eps = 1, it's Random, when eps = 0, it's Greedy
        if np.random.uniform() <= eps:
            # exploration: random allocation
            a_index = np.random.choice(feasible_choices)
            t_index = max_sys_improvement_values[a_index][0]
        else:
            # exploitation: allocationelse based on reputation or efficiency
            best_sys_improvement_value = max(max_sys_improvement_values.items(), key=lambda x: x[1][1])
            a_index, t_index = best_sys_improvement_value[0], best_sys_improvement_value[1][0]

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != task_num:
            affected_a_indexes.extend(coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for n_id in coalition_structure[t_index]:
                task_cons[n_id][t_index] = agent_contribution(agents, tasks, n_id, t_index, coalition_structure[t_index], constraints, gamma)
                cur_con[n_id] = task_cons[n_id][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != task_num):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for n_id in coalition_structure[old_t_index]:
                task_cons[n_id][old_t_index] = agent_contribution(agents, tasks, n_id, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[n_id] = task_cons[n_id][old_t_index]

        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for n_id in range(0, agent_num):
                if (n_id not in coalition_structure[t_ind]) and (t_ind in a_taskInds[n_id]):
                    task_cons[n_id][t_ind] = agent_contribution(agents, tasks, n_id, t_ind, coalition_structure[t_ind], constraints, gamma)

        # update node values
        for n_id, n_value in deflect_nodes_alt_value_info[a_index].items():
            node_value_info[n_id] = n_value

        for n_id, n_value in added_nodes_alt_value_info[a_index][t_index].items():
            node_value_info[n_id] = n_value


        info_get_cur_con = {
            i: get_cur_con_tree(i, node_type_info, children_info, parent_info, task2leaf, allocation_structure, node_value_info, cur_con)
            for i in range(agent_num)
        }

        deflect_nodes_alt_value_info = {
            i: info_get_cur_con[i][0]
            for i in range(agent_num)
        }

        value_lost_info = {
            i: info_get_cur_con[i][1]
            for i in range(agent_num)
        }

        info_get_move_val_tree = {
            i: {
                j: get_move_val_tree(i, j, deflect_nodes_alt_value_info[i], node_type_info, parent_info, task2leaf, node_value_info, task_cons, root_node_id)
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }

        added_nodes_alt_value_info = {
            i: {
                j: info_get_move_val_tree[i][j][0]
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }

        value_added_info = {
            i: {
                j: info_get_move_val_tree[i][j][1]
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }

        sys_improvement_values_info = {
            i: {
                j: value_added_info[i][j] - value_lost_info[i]
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }

        max_sys_improvement_values = {
            i: max(sys_improvement_values_info[i].items(), key=lambda x: x[1])
            for i in range(agent_num)
        }


    return (
        coalition_structure,
        node_value_info,
        iteration_count,
        re_assignment_count,
    )


def treeGNE2(
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]], 
        parent_info: dict[int, int],
        task2leaf: list[int],
        leaf2task: dict[int, int],
        tasks: list[list[int]],
        agents: list[dict[int, float]],
        constraints,
        coalition_structure : list[list[int]] = [],
        selected_tasks : list[int] = None,
        eps=0, 
        gamma=1,
        root_node_id=0,
    ):
    
    """
    GreedyNE on an AND-OR tree.

    At each iteration, calculate the change in nodes values when an agent defects from its current task, and the change in nodes values when an agent moves to a new task.
    """
    re_assignment_count = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)

    if selected_tasks is None:
        selected_tasks = list(range(len(tasks)))
        task_selected = [True for i in range(len(tasks))]

    else:    
        task_selected = [False for i in range(len(tasks))]
        for j in selected_tasks:
            task_selected[j] = True

    allocation_structure = [task_num for i in range(0, agent_num)]  # each indicate the current task that agent i is allocated to, if = N, means not allocated
    if coalition_structure is None or coalition_structure == []:
        coalition_structure = [[] for j in range(0, task_num)] + [list(range(0, agent_num))]  # default coalition structure, the last one is dummy coalition
        cur_con = [0 for j in range(0, agent_num)]
    else:

        if len(coalition_structure) < task_num:
            coalition_structure.append([])

        for j in range(0, task_num):
            if not task_selected[j]:
                coalition_structure[len(task_num)] += coalition_structure[j]
                coalition_structure[j] = []

        for j in range(0, task_num):
            for n_id in coalition_structure[j]:
                allocation_structure[n_id] = j

        cur_con = [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j != task_num and task_selected[j]
            else 0
            for i, j in enumerate(allocation_structure)
        ]

    task_cons = [
        [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j in a_taskInds[i] and task_selected[j]
            else float("-inf")
            for j in range(0, task_num)
        ] + [0]
        for i in range(0, agent_num)
    ]

    move_vals = {
        i : {
            j : task_cons[i][j] - cur_con[i]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }


    node_value_info = get_node_value_info(node_type_info, children_info, leaf2task, coalition_structure, root_node_id)

    info_get_cur_con = {
        i: get_cur_con_tree(i, node_type_info, children_info, parent_info, task2leaf, allocation_structure, node_value_info, cur_con, root_node_id)
        for i in range(agent_num)
    }

    deflect_nodes_alt_value_info = {
        i: info_get_cur_con[i][0]
        for i in range(agent_num)
    }

    value_lost_info = {
        i: info_get_cur_con[i][1]
        for i in range(agent_num)
    }

    info_get_move_val_tree = {
        i: {
            j: get_move_val_tree(i, j, deflect_nodes_alt_value_info[i], node_type_info, parent_info, task2leaf, node_value_info, task_cons, root_node_id)
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }

    added_nodes_alt_value_info = {
        i: {
            j: info_get_move_val_tree[i][j][0]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }

    value_added_info = {
        i: {
            j: info_get_move_val_tree[i][j][1]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }

    sys_improvement_values_info = {
        i: {
            j: value_added_info[i][j] - value_lost_info[i]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }


    iteration_count = 0
    while True:
        iteration_count += 1
        feasible_choices_map = {
            i : [
                j for j in a_taskInds[i] if task_selected[j] and (
                    (sys_improvement_values_info[i][j] > 0) or (sys_improvement_values_info[i][j] == 0 and move_vals[i][j] > 0)
                )
            ]
            for i in range(0, agent_num)
        }
        feasible_choices = [(i, j) for i in feasible_choices_map for j in feasible_choices_map[i]]
        if len(feasible_choices) == 0:
            break  # reach NE solution
        # when eps = 1, it's Random, when eps = 0, it's Greedy
        if np.random.uniform() <= eps:
            # exploration: random allocation
            a_index, t_index = np.random.choice(feasible_choices)
        else:
            # exploitation: allocationelse based on reputation or efficiency
            a_index, t_index = max(feasible_choices, key=lambda x: (sys_improvement_values_info[x[0]][x[1]], move_vals[x[0]][x[1]]))
            

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != task_num:
            affected_a_indexes.extend(coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for n_id in coalition_structure[t_index]:
                task_cons[n_id][t_index] = agent_contribution(agents, tasks, n_id, t_index, coalition_structure[t_index], constraints, gamma)
                cur_con[n_id] = task_cons[n_id][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != task_num):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for i in coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        for i in affected_a_indexes:
            move_vals[i] = [
                task_cons[i][j] - cur_con[i]
                if (j in a_taskInds[i] and task_selected[j]) or j == task_num
                else float("-inf")
                for j in range(0, task_num + 1)
            ]

        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for i in range(0, agent_num):
                if (i not in coalition_structure[t_ind]) and (t_ind in a_taskInds[i]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, coalition_structure[t_ind], constraints, gamma)
                    move_vals[i][t_ind] = task_cons[i][t_ind] - cur_con[i]

        # update node values
        for n_id, n_value in deflect_nodes_alt_value_info[a_index].items():
            node_value_info[n_id] = n_value

        for n_id, n_value in added_nodes_alt_value_info[a_index][t_index].items():
            node_value_info[n_id] = n_value


        info_get_cur_con = {
            i: get_cur_con_tree(i, node_type_info, children_info, parent_info, task2leaf, allocation_structure, node_value_info, cur_con)
            for i in range(agent_num)
        }

        deflect_nodes_alt_value_info = {
            i: info_get_cur_con[i][0]
            for i in range(agent_num)
        }

        value_lost_info = {
            i: info_get_cur_con[i][1]
            for i in range(agent_num)
        }

        info_get_move_val_tree = {
            i: {
                j: get_move_val_tree(i, j, deflect_nodes_alt_value_info[i], node_type_info, parent_info, task2leaf, node_value_info, task_cons, root_node_id)
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }

        added_nodes_alt_value_info = {
            i: {
                j: info_get_move_val_tree[i][j][0]
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }

        value_added_info = {
            i: {
                j: info_get_move_val_tree[i][j][1]
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }

        sys_improvement_values_info = {
            i: {
                j: value_added_info[i][j] - value_lost_info[i]
                for j in a_taskInds[i] if task_selected[j]
            }
            for i in range(agent_num)
        }



    return (
        coalition_structure,
        node_value_info,
        iteration_count,
        re_assignment_count,
    )


def fastTreeGNE2(
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]], 
        parent_info: dict[int, int],
        task2leaf: list[int],
        leaf2task: dict[int, int],
        tasks: list[list[int]],
        agents: list[dict[int, float]],
        constraints,
        coalition_structure : list[list[int]] = [],
        selected_tasks : list[int] = None,
        gamma=1,
        root_node_id=0,
    ):
    
    """
    GreedyNE on an AND-OR tree.

    At each iteration, calculate the change in nodes values when an agent defects from its current task, and the change in nodes values when an agent moves to a new task.

    Quickly and greedily find the best move by benchmarking (bounding) the added value for each node.
    """
    re_assignment_count = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)

    if selected_tasks is None:
        selected_tasks = list(range(len(tasks)))
        task_selected = [True for i in range(len(tasks))]

    else:    
        task_selected = [False for i in range(len(tasks))]
        for j in selected_tasks:
            task_selected[j] = True

    allocation_structure = [task_num for i in range(0, agent_num)]  # each indicate the current task that agent i is allocated to, if = N, means not allocated
    if coalition_structure is None or coalition_structure == []:
        coalition_structure = [[] for j in range(0, task_num)] + [list(range(0, agent_num))]  # default coalition structure, the last one is dummy coalition
        cur_con = [0 for j in range(0, agent_num)]
    else:

        if len(coalition_structure) < task_num:
            coalition_structure.append([])

        for j in range(0, task_num):
            if not task_selected[j]:
                coalition_structure[len(task_num)] += coalition_structure[j]
                coalition_structure[j] = []

        for j in range(0, task_num):
            for n_id in coalition_structure[j]:
                allocation_structure[n_id] = j

        cur_con = [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j != task_num and task_selected[j]
            else 0
            for i, j in enumerate(allocation_structure)
        ]

    task_cons = [
        [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j in a_taskInds[i] and task_selected[j]
            else float("-inf")
            for j in range(0, task_num)
        ] + [0]
        for i in range(0, agent_num)
    ]

    move_vals = {
        i : {
            j : task_cons[i][j] - cur_con[i]
            for j in a_taskInds[i] if task_selected[j]
        }
        for i in range(agent_num)
    }


    node_value_info = get_node_value_info(node_type_info, children_info, leaf2task, coalition_structure, root_node_id)

    info_get_cur_con = {
        i: get_cur_con_tree(i, node_type_info, children_info, parent_info, task2leaf, allocation_structure, node_value_info, cur_con, root_node_id)
        for i in range(agent_num)
    }

    deflect_nodes_alt_value_info = {
        i: info_get_cur_con[i][0]
        for i in range(agent_num)
    }

    value_lost_info = {
        i: info_get_cur_con[i][1]
        for i in range(agent_num)
    }

    added_nodes_alt_value_info = {
        i: { }
        for i in range(agent_num)
    }


    best_move_agent, best_move_task = 0, 0
    best_improvement_value = float("-inf")
    best_move_move_val = float("-inf")
    for i in range(agent_num):
        best_added_value = best_improvement_value + value_lost_info[i]
        for j in a_taskInds[i]:
            if not task_selected[j]:
                continue
            added_nodes_alt_values, value_added = get_move_val_tree(i, j, deflect_nodes_alt_value_info[i], node_type_info, parent_info, task2leaf, node_value_info, task_cons, root_node_id, value_added_benchmark=best_added_value)
            move_val = move_vals[i][j]
            if value_added > best_added_value or (value_added == best_added_value and move_val > best_move_move_val):
                added_nodes_alt_value_info[i][j] = added_nodes_alt_values
                best_added_value = value_added
                best_improvement_value = value_added - value_lost_info[i]
                best_move_move_val = move_val
                best_move_agent, best_move_task = i, j


    iteration_count = 0
    while True:
        iteration_count += 1
        if not ((best_improvement_value > 0) or (best_improvement_value == 0 and best_move_move_val > 0)):
            break

        a_index, t_index = best_move_agent, best_move_task            

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != task_num:
            affected_a_indexes.extend(coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for n_id in coalition_structure[t_index]:
                task_cons[n_id][t_index] = agent_contribution(agents, tasks, n_id, t_index, coalition_structure[t_index], constraints, gamma)
                cur_con[n_id] = task_cons[n_id][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != task_num):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for i in coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        for i in affected_a_indexes:
            move_vals[i] = [
                task_cons[i][j] - cur_con[i]
                if (j in a_taskInds[i] and task_selected[j]) or j == task_num
                else float("-inf")
                for j in range(0, task_num + 1)
            ]

        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for i in range(0, agent_num):
                if (i not in coalition_structure[t_ind]) and (t_ind in a_taskInds[i]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, coalition_structure[t_ind], constraints, gamma)
                    move_vals[i][t_ind] = task_cons[i][t_ind] - cur_con[i]

        # update node values
        for n_id, n_value in deflect_nodes_alt_value_info[a_index].items():
            node_value_info[n_id] = n_value

        for n_id, n_value in added_nodes_alt_value_info[a_index][t_index].items():
            node_value_info[n_id] = n_value


        info_get_cur_con = {
            i: get_cur_con_tree(i, node_type_info, children_info, parent_info, task2leaf, allocation_structure, node_value_info, cur_con)
            for i in range(agent_num)
        }

        deflect_nodes_alt_value_info = {
            i: info_get_cur_con[i][0]
            for i in range(agent_num)
        }

        value_lost_info = {
            i: info_get_cur_con[i][1]
            for i in range(agent_num)
        }

        added_nodes_alt_value_info = {
            i: { }
            for i in range(agent_num)
        }

        best_move_agent, best_move_task = 0, 0
        best_improvement_value = float("-inf")
        best_move_move_val = float("-inf")
        for i in range(agent_num):
            best_added_value = best_improvement_value + value_lost_info[i]
            for j in a_taskInds[i]:
                if not task_selected[j]:
                    continue
                added_nodes_alt_values, value_added = get_move_val_tree(i, j, deflect_nodes_alt_value_info[i], node_type_info, parent_info, task2leaf, node_value_info, task_cons, root_node_id, value_added_benchmark=best_added_value)
                move_val = move_vals[i][j]
                if value_added > best_added_value or (value_added == best_added_value and move_val > best_move_move_val):
                    added_nodes_alt_value_info[i][j] = added_nodes_alt_values
                    best_added_value = value_added
                    best_improvement_value = value_added - value_lost_info[i]
                    best_move_move_val = move_val
                    best_move_agent, best_move_task = i, j


    return (
        coalition_structure,
        node_value_info,
        iteration_count,
        re_assignment_count,
    )



In [17]:


def aGreedyNE(
        agents : list[list[float]], 
        tasks : list[list[int]],
        constraints : tuple[list[list[int]], list[list[int]]],
        coalition_structure : list[list[int]] = [],
        selected_tasks : list[int] = None,
        selected_agents : list[int] = None,
        eps=0, 
        gamma=1
    ):
    """
    GreedyNE on a subset of tasks.

    The target is to focus all agents on working on only the selected tasks.
    """
    re_assignment_count = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)

    if selected_tasks is None:
        selected_tasks = list(range(len(tasks)))
        task_selected = [True for j in range(len(tasks))]

    else:    
        task_selected = [False for j in range(len(tasks))]
        for j in selected_tasks:
            task_selected[j] = True

    task_selected.append(True) # dummy task


    if selected_agents is None:
        selected_agents = list(range(0, agent_num))
        agent_selected = [True for i in range(len(agents))]

    else:
        agent_selected = [False for i in range(len(agents))]
        for i in selected_agents:
            agent_selected[i] = True


    allocation_structure = { i : task_num for i in selected_agents }
    # each indicate the current task that agent i is allocated to, if = N, means not allocated
    
    if coalition_structure is None or coalition_structure == []:
        coalition_structure = [[] for j in range(0, task_num)] + [list(range(0, agent_num))]  # default coalition structure, the last one is dummy coalition
        cur_con = { i : 0 for i in selected_agents }
    else:

        if len(coalition_structure) < task_num:
            coalition_structure.append([])

        for j in range(0, task_num):
            if not task_selected[j]:
                coalition_structure[task_num] += coalition_structure[j]
                coalition_structure[j] = []

        for j in range(0, task_num):
            for i in coalition_structure[j]:
                allocation_structure[i] = j

        cur_con = {
            i : agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j != task_num and task_selected[j]
            else 0
            for i, j in allocation_structure.items()
        }
        

    task_cons = {
        i : {
            j : agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j != task_num and task_selected[j]
            else 0 if j == task_num
            else float("-inf")
            for j in a_taskInds[i] + [task_num]
        }
        for i in selected_agents
    }
    # the last 0 indicate not allocated

    move_vals = {
        i : {
            j : task_cons[i][j] - cur_con[i] 
            if j == task_num or task_selected[j]
            else float("-inf")
            for j in a_taskInds[i] + [task_num]
        }
        for i in selected_agents
    }

    max_moves = {
        i : max(move_vals[i].items(), key=lambda x: x[1])
        for i in selected_agents
    }

    iteration_count = 0
    while True:
        iteration_count += 1
        feasible_choices = [i for i in selected_agents if max_moves[i][1] > 0]
        if feasible_choices == []:
            break  # reach NE solution
        # when eps = 1, it's Random, when eps = 0, it's Greedy
        if np.random.uniform() <= eps:
            # exploration: random allocation
            a_index = np.random.choice(feasible_choices)
            t_index = max_moves[a_index][0]
        else:
            # exploitation: allocationelse based on reputation or efficiency
            best_move = max(max_moves.items(), key=lambda x: x[1][1])
            a_index = best_move[0]
            t_index = best_move[1][0]

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != task_num:
            affected_a_indexes.extend(coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for i in coalition_structure[t_index]:
                if agent_selected[i]:
                    task_cons[i][t_index] = agent_contribution(agents, tasks, i, t_index, coalition_structure[t_index], constraints, gamma)
                    cur_con[i] = task_cons[i][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != task_num):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for i in coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        for i in affected_a_indexes:
            move_vals[i] = {
                j : task_cons[i][j] - cur_con[i]
                if j == task_num or task_selected[j]
                else float("-inf")
                for j in a_taskInds[i] + [task_num]
            }


        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for i in range(0, agent_num):
                if (i not in coalition_structure[t_ind]) and (t_ind in a_taskInds[i]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, coalition_structure[t_ind], constraints, gamma)
                    move_vals[i][t_ind] = task_cons[i][t_ind] - cur_con[i]

        max_moves = {
            i : max(move_vals[i].items(), key=lambda x: x[1])
            for i in selected_agents
        }


    return (
        coalition_structure,
        sys_rewards_tasks(tasks, agents, coalition_structure, gamma),
        iteration_count,
        re_assignment_count,
    )


def adGreedyNE(
        agents : list[list[float]], 
        tasks : list[list[int]],
        constraints : tuple[list[list[int]], list[list[int]]],
        original_coalition_structure : dict[int, list[int]] = None,
        selected_tasks : list[int] = None,
        selected_agents : list[int] = None,
        eps=0, 
        gamma=1,
        dummy_task_id = None
    ):
    """
    GreedyNE on a subset of tasks.

    The target is to focus all agents on working on only the selected tasks and skip all other tasks.
    """
    re_assignment_count = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)

    if dummy_task_id is None:
        dummy_task_id = len(tasks)

    task_selected = {}
    if selected_tasks is None:
        selected_tasks = list(range(len(tasks)))
        task_selected = { j : True for j in range(len(tasks)) }
    else:    
        task_selected = { j : False for j in range(len(tasks)) }
        for j in selected_tasks:
            task_selected[j] = True
    task_selected[dummy_task_id] = True


    if selected_agents is None:
        selected_agents = list(range(0, agent_num))
        agent_selected = { i : True for i in range(len(agents)) }

    else:
        agent_selected = { i : True for i in range(len(agents)) }
        for i in selected_agents:
            agent_selected[i] = True

    new_coalition_structure = { j : [] for j in selected_tasks }

    new_coalition_structure[dummy_task_id] = []

    if original_coalition_structure is None or original_coalition_structure == {}:
        original_coalition_structure = {}
        new_coalition_structure[dummy_task_id] = list(range(0, agent_num))

    
    for j in range(0, task_num):
        if task_selected[j]:
            new_coalition_structure[j] = [i for i in original_coalition_structure.get(j, []) if agent_selected[i]]
        else:
            new_coalition_structure[dummy_task_id] += [i for i in original_coalition_structure.get(j, []) if agent_selected[i]]


    allocation_structure = {}
    cur_con = {}

    # for j in new_coalition_structure:
    #     for i in new_coalition_structure[j]:
    #         allocation_structure[i] = j
    #         if j != task_num:
    #             cur_con[i] = agent_contribution(agents, tasks, i, j, new_coalition_structure[j], constraints, gamma)
    #         else:
    #             cur_con[i] = 0

    allocation_structure = { i : j for j in new_coalition_structure for i in new_coalition_structure[j] }

    cur_con = { i : agent_contribution(agents, tasks, i, j, new_coalition_structure[j], constraints, gamma) if j != dummy_task_id else 0 for i, j in allocation_structure.items() }
        

    task_cons = {
        i : {
            j : agent_contribution(agents, tasks, i, j, new_coalition_structure[j], constraints, gamma)
            if j != dummy_task_id and task_selected[j]
            else 0 if j == dummy_task_id
            else float("-inf")
            for j in a_taskInds[i] + [dummy_task_id]
        }
        for i in selected_agents
    }
    # the last 0 indicate not allocated

    move_vals = {
        i : {
            j : task_cons[i][j] - cur_con[i] 
            if j == dummy_task_id or task_selected[j]
            else float("-inf")
            for j in a_taskInds[i] + [dummy_task_id]
        }
        for i in selected_agents
    }

    max_moves = {
        i : max(move_vals[i].items(), key=lambda x: x[1])
        for i in selected_agents
    }

    iteration_count = 0
    while True:
        iteration_count += 1
        feasible_choices = [i for i in selected_agents if max_moves[i][1] > 0]
        if feasible_choices == []:
            break  # reach NE solution
        # when eps = 1, it's Random, when eps = 0, it's Greedy
        if np.random.uniform() <= eps:
            # exploration: random allocation
            a_index = np.random.choice(feasible_choices)
            t_index = max_moves[a_index][0]
        else:
            # exploitation: allocationelse based on reputation or efficiency
            best_move = max(max_moves.items(), key=lambda x: x[1][1])
            a_index = best_move[0]
            t_index = best_move[1][0]

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        new_coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != dummy_task_id:
            affected_a_indexes.extend(new_coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for i in new_coalition_structure[t_index]:
                if agent_selected[i]:
                    task_cons[i][t_index] = agent_contribution(agents, tasks, i, t_index, new_coalition_structure[t_index], constraints, gamma)
                    cur_con[i] = task_cons[i][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != dummy_task_id):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            new_coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(new_coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for i in new_coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, new_coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        for i in affected_a_indexes:
            move_vals[i] = {
                j : task_cons[i][j] - cur_con[i]
                if j == dummy_task_id or task_selected[j]
                else float("-inf")
                for j in a_taskInds[i] + [dummy_task_id]
            }


        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for i in range(0, agent_num):
                if (i not in new_coalition_structure[t_ind]) and (t_ind in a_taskInds[i]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, new_coalition_structure[t_ind], constraints, gamma)
                    move_vals[i][t_ind] = task_cons[i][t_ind] - cur_con[i]

        max_moves = {
            i : max(move_vals[i].items(), key=lambda x: x[1])
            for i in selected_agents
        }


    return (
        new_coalition_structure,
        sum(
            task_reward(tasks[j], [agents[i] for i in new_coalition_structure[j]], gamma)
            for j in selected_tasks
        ),
        iteration_count,
        re_assignment_count,
    )


def alGreedyNE(
        agents : list[list[float]], 
        tasks : list[list[int]],
        constraints : tuple[list[list[int]], list[list[int]]],
        original_allocation_structure : dict[int, int] = None,
        selected_tasks : list[int] = None,
        selected_agents : list[int] = None,
        eps=0, 
        gamma=1,
        dummy_task_id = None
    ):
    """
    GreedyNE on a subset of tasks.

    The target is to focus all agents on working on only the selected tasks and skip all other tasks.
    """
    re_assignment_count = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)

    if dummy_task_id is None:
        dummy_task_id = len(tasks)

    task_selected = {}
    if selected_tasks is None:
        selected_tasks = list(range(len(tasks)))
        task_selected = { j : True for j in range(len(tasks)) }
    else:    
        task_selected = { j : False for j in range(len(tasks)) }
        for j in selected_tasks:
            task_selected[j] = True
    task_selected[dummy_task_id] = True


    if selected_agents is None:
        selected_agents = list(range(0, agent_num))
        agent_selected = { i : True for i in range(len(agents)) }

    else:
        agent_selected = { i : True for i in range(len(agents)) }
        for i in selected_agents:
            agent_selected[i] = True


    if original_allocation_structure is None or original_allocation_structure == {}:
        original_allocation_structure = { i : dummy_task_id for i in selected_agents }


    allocation_structure = { i : dummy_task_id for i in selected_agents }
    for i, j in original_allocation_structure.items():
        if agent_selected[i]:
            allocation_structure[i] = j

    coalition_structure = { j : [] for j in selected_tasks + [dummy_task_id] }

    cur_con = { i : 0 for i in selected_agents }

    for i, j in allocation_structure.items():
        if task_selected[j]:
            coalition_structure[j].append(i)
            if j != dummy_task_id:
                cur_con[i] = agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            else:
                cur_con[i] = 0
        else:
            coalition_structure[dummy_task_id].append(i)
            allocation_structure[i] = dummy_task_id
            cur_con[i] = 0

        

    task_cons = {
        i : {
            j : agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j != dummy_task_id and task_selected[j]
            else 0 if j == dummy_task_id
            else float("-inf")
            for j in a_taskInds[i] + [dummy_task_id]
        }
        for i in selected_agents
    }
    # the last 0 indicate not allocated

    move_vals = {
        i : {
            j : task_cons[i][j] - cur_con[i] 
            if j == dummy_task_id or task_selected[j]
            else float("-inf")
            for j in a_taskInds[i] + [dummy_task_id]
        }
        for i in selected_agents
    }


    max_moves = {
        i : max(move_vals[i].items(), key=lambda x: x[1])
        for i in selected_agents
    }

    iteration_count = 0
    while True:
        iteration_count += 1
        feasible_choices = [i for i in selected_agents if max_moves[i][1] > 0]
        if feasible_choices == []:
            break  # reach NE solution
        # when eps = 1, it's Random, when eps = 0, it's Greedy
        if np.random.uniform() <= eps:
            # exploration: random allocation
            a_index = np.random.choice(feasible_choices)
            t_index = max_moves[a_index][0]
        else:
            # exploitation: allocationelse based on reputation or efficiency
            best_move = max(max_moves.items(), key=lambda x: x[1][1])
            a_index = best_move[0]
            t_index = best_move[1][0]

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != dummy_task_id:
            affected_a_indexes.extend(coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for i in coalition_structure[t_index]:
                if agent_selected[i]:
                    task_cons[i][t_index] = agent_contribution(agents, tasks, i, t_index, coalition_structure[t_index], constraints, gamma)
                    cur_con[i] = task_cons[i][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != dummy_task_id):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for i in coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        for i in affected_a_indexes:
            move_vals[i] = {
                j : task_cons[i][j] - cur_con[i]
                if j == dummy_task_id or task_selected[j]
                else float("-inf")
                for j in a_taskInds[i] + [dummy_task_id]
            }


        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for i in selected_agents:
                if (i not in coalition_structure[t_ind]) and (t_ind in a_taskInds[i]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, coalition_structure[t_ind], constraints, gamma)
                    move_vals[i][t_ind] = task_cons[i][t_ind] - cur_con[i]

        max_moves = {
            i : max(move_vals[i].items(), key=lambda x: x[1])
            for i in selected_agents
        }


    return (
        coalition_structure,
        allocation_structure,
        sum(
            task_reward(tasks[j], [agents[i] for i in coalition_structure[j]], gamma)
            for j in selected_tasks
        ),
        iteration_count,
        re_assignment_count,
    )


In [18]:

def ao_search(
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]],
        reward_function: dict[int, float],
        root_node_id=0,
    ):
    
    
    def ao_helper(node_id: int):

        node_type = node_type_info[node_id]
        
        if node_type == NodeType.LEAF:
            return reward_function[node_id], [node_id]

        total_reward = 0 if node_type == NodeType.AND else float('-inf')

        best_solution = []

        if node_type == NodeType.AND:

            for child_id in children_info[node_id]:

                child_reward, child_solution = ao_helper(child_id)

                total_reward += child_reward
                best_solution += child_solution
                
        else:
            for child_id in children_info[node_id]:
                
                child_reward, child_solution = ao_helper(child_id)

                if child_reward > total_reward:
                    total_reward = child_reward
                    best_solution = child_solution
                    
        # expanded.append(node_id)
        return total_reward, best_solution
    
    total_reward, best_leafs_solution = ao_helper(root_node_id)
    
    return total_reward, best_leafs_solution



In [19]:

def simpleGNE(
        node_type_info: dict[int, NodeType],
        children_info: dict[int, list[int]],
        leaf2task: dict[int, int],
        agents : list[list[float]], 
        tasks : list[list[int]],
        constraints : tuple[list[list[int]], list[list[int]]],
        coalition_structure : list[list[int]] = [],
        selected_tasks : list[int] = None,
        root_node_id=0,
        eps=0, 
        gamma=1
    ):
    prev_sys_reward = 0
    true_sys_reward = float('inf')
    iteration_count_1 = 0
    re_assignment_count_1 = 0
    iteration_count_2 = 0
    re_assignment_count_2 = 0
    total_loop_count = 0
    while True:

        total_loop_count += 1

        coalition_structure, system_reward, iteration_count, re_assignment_count = aGreedyNE(
            agents=agents,
            tasks=tasks,
            constraints=constraints,
            coalition_structure=coalition_structure,
            selected_tasks=selected_tasks,
            eps=eps,
            gamma=gamma
        )

        iteration_count_1 += iteration_count
        re_assignment_count_1 += re_assignment_count

        reward_function = {
            leaf_id: task_reward(tasks[leaf2task[leaf_id]], [agents[i] for i in coalition_structure[leaf2task[leaf_id]]], gamma)
            for leaf_id in leaf2task
        }
        
        true_sys_reward, best_leafs_solution = ao_search(node_type_info, children_info, reward_function, root_node_id)

        new_selected_tasks = [leaf2task[leaf_id] for leaf_id in best_leafs_solution]

        coalition_structure, true_sys_reward, iteration_count, re_assignment_count = aGreedyNE(
            agents=agents,
            tasks=tasks,
            constraints=constraints,
            coalition_structure=coalition_structure,
            selected_tasks=new_selected_tasks,
            eps=eps,
            gamma=gamma
        )

        iteration_count_2 += iteration_count
        re_assignment_count_2 += re_assignment_count

        if true_sys_reward <= prev_sys_reward:
            break

        prev_sys_reward = true_sys_reward

    return (
        coalition_structure,
        true_sys_reward,
        iteration_count_1,
        re_assignment_count_1,
        iteration_count_2,
        re_assignment_count_2,
        total_loop_count
    )

In [20]:



def get_upperbound_node_descendants(
        query_nodeId : int,
        agents_group : list[int],
        ubcv_info : dict[int, np.ndarray],
        children_info : dict[int, list[int]],
        node_type_info : dict[int, NodeType],
        capabilities : list[int], 
        agents : list[list[float]],
        nodes_constraints : tuple[list[list[int]], dict[int, list[int]]],
    ):
    """
    Calculate the upper bound of the reward (utility) at each descendant of the queried node in the AND-OR goal tree, including the queried node itself.
    
    Calculate the upper bound of the reward (utility) at each node of the AND-OR goal tree.

    Refine the upper bound by taking the minimum of the upper bound calculated from the children nodes, and the upper bound calculated from the current node.

    """

    agent_selected = {i: False for i in range(len(agents))}
    for i in agents_group:
        agent_selected[i] = True

    def _upperbound_node(node_id):
        """
        Calculate the upper bound of the system reward, i.e. at the root of the AND-OR goal tree.
        """
        nodes_agents = nodes_constraints[1]

        caps_ranked = [sorted([agents[i][c] for i in nodes_agents[node_id] if agent_selected[i]], reverse=True) for c in capabilities]

        cap_req_num = ubcv_info[node_id]
        
        return sum([sum(caps_ranked[c][:int(cap_req_num[c])]) for c in capabilities])

    descendant_nodes = list(traverse_tree(children_info, root_node_id=query_nodeId))
    
    nodes_upper_bound = { node_id : _upperbound_node(node_id) for node_id in descendant_nodes }

    nodes_upper_bound_min = { node_id : 0 for node_id in descendant_nodes }

    def _min_upper_bound(node_id : int):
        node_type = node_type_info[node_id]

        if node_type == NodeType.LEAF:
            nodes_upper_bound_min[node_id] = nodes_upper_bound[node_id]

        elif node_type == NodeType.OR:
            nodes_upper_bound_min[node_id] = max(_min_upper_bound(child_id) for child_id in children_info[node_id])

        elif node_type == NodeType.AND:
            nodes_upper_bound_min[node_id] = sum(_min_upper_bound(child_id) for child_id in children_info[node_id])

        else:
            raise Exception("Unsupported node type")

        nodes_upper_bound_min[node_id] = min(nodes_upper_bound[node_id], nodes_upper_bound_min[node_id])
        return nodes_upper_bound_min[node_id]
        
    _min_upper_bound(query_nodeId)

    return nodes_upper_bound_min




In [21]:


def ao_search_2(
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]],
        reward_function: dict[int, float],
        root_node_id=0,
    ):
    
    visited = {}
    # expanded = []
    st_children_info : dict[int, list[int]] = {
        node_id: [] if node_type_info[node_id] == NodeType.OR else children_list
        for node_id, children_list in children_info.items()
    }

    def aos_helper(node_id: int):

        if node_id not in visited:
            visited[node_id] = True            

        node_type = node_type_info[node_id]
        
        if node_type == NodeType.LEAF:
            return reward_function[node_id], [node_id]

        total_reward = 0 if node_type == NodeType.AND else float('-inf')

        best_solution = []

        if node_type == NodeType.AND:

            for child_id in children_info[node_id]:

                child_reward, child_solution = aos_helper(child_id)

                total_reward += child_reward
                best_solution += child_solution
                
        else:
            for child_id in children_info[node_id]:

                if st_children_info[node_id] == []:
                    st_children_info[node_id] = [child_id]
                
                child_reward, child_solution = aos_helper(child_id)

                if child_reward > total_reward:
                    total_reward = child_reward
                    best_solution = child_solution
                    st_children_info[node_id] = [child_id]

                    
        # expanded.append(node_id)
        return total_reward, best_solution
    
    total_reward, best_leafs_solution = aos_helper(root_node_id)
    
    return total_reward, best_leafs_solution, st_children_info





In [22]:

def BnBOrNE(
        node_type_info: dict[int, NodeType], 
        children_info: dict[int, list[int]], 
        ubcv_info : dict[int, np.ndarray],
        leaf2task: dict[int, int],
        task2leaf: dict[int, int],
        leaves_list_info: dict[int, list[int]],
        capabilities: list[int],
        tasks: list[list[int]],
        agents: list[dict[int, float]],
        constraints,
        nodes_constraints : tuple[list[list[int]], dict[int, list[int]]],
        coalition_structure : dict[int, list[int]] = {},
        eps=0, 
        gamma=1,
        root_node_id=0,
        use_branch_and_bound=True,
        skip_initial_branch=False,
    ):
    
    task_num = len(tasks)
    agent_num = len(agents)

    def _GreedyNE(original_allocation_structure, selected_tasks, selected_agents):
        return alGreedyNE(
            agents=agents, 
            tasks=tasks, 
            constraints=constraints, 
            original_allocation_structure=original_allocation_structure,
            selected_tasks=selected_tasks, 
            selected_agents=selected_agents, 
            eps=eps, 
            gamma=gamma
        )
    
    def _ao_search(reward_function, root_node_id):
        return ao_search_2(
            node_type_info=node_type_info,
            children_info=children_info,
            reward_function=reward_function,
            root_node_id=root_node_id,
        )
    

    def _get_upper_bound(node_id, agents_group):
        return get_upperbound_node_descendants(
            query_nodeId=node_id,
            agents_group=agents_group,
            ubcv_info=ubcv_info,
            children_info=children_info,
            node_type_info=node_type_info,
            capabilities=capabilities,
            agents=agents,
            nodes_constraints=nodes_constraints,
        )

    
    if coalition_structure is None or coalition_structure == {}:
        coalition_structure = {j: [] for j in range(0, len(tasks))}
        coalition_structure[len(tasks)] = list(range(0, len(agents)))  # default coalition structure, the last one is dummy coalition

    allocation_structure_global = {i: task_num for i in range(0, agent_num)}
    for j in coalition_structure:
        for i in coalition_structure[j]:
            allocation_structure_global[i] = j

    t_agents = constraints[1]


    def aos_helper(node_id, agents_group, allocation_structure_0 = None):

        total_iterations_count = 0
        total_reassignment_count = 0

        node_type = node_type_info[node_id]

        if node_type == NodeType.LEAF:
            task_id = leaf2task[node_id]
            new_allocation_structure = { i: task_id if i in t_agents[task_id] else task_num for i in agents_group }
            reward_value = task_reward(tasks[task_id], [agents[i] for i in agents_group if i in t_agents[task_id]], gamma)
            return new_allocation_structure, reward_value, total_iterations_count, total_reassignment_count


        descendant_leaves = leaves_list_info[node_id]
        descendant_tasks = [leaf2task[task_id] for task_id in descendant_leaves]

        # Initialization phase

        # Initialize allocation structure
        not_initialized = False
        if allocation_structure_0 is None:
            not_initialized = True
        else:
            not_initialized = True
            for i in agents_group:
                if i in allocation_structure_0 and allocation_structure_0[i] in descendant_tasks:
                    not_initialized = False
                    break

        if not_initialized:
            # Perform GreedyNE on the entire system, not considering the tree structure
            coalition_structure_1, allocation_structure_1, _, iter_count_1, reassign_1 = _GreedyNE(
                original_allocation_structure=None,
                selected_tasks=descendant_tasks,
                selected_agents=agents_group,
            )
        else:
            allocation_structure_1 = allocation_structure_0
            coalition_structure_1 = { j: [] for j in range(0, task_num + 1) }
            for i in allocation_structure_1:
                coalition_structure_1[allocation_structure_1[i]].append(i)
                iter_count_1 = 0
                reassign_1 = 0
                    
        # Initialization phase

        reward_function = {
            task2leaf[task_id]: task_reward(tasks[task_id], [agents[i] for i in coalition_structure_1[task_id]], gamma)
            for task_id in descendant_tasks
        }

        true_sys_reward_1, best_leafs_solution, st_children_info = _ao_search(
            reward_function=reward_function,
            root_node_id=node_id,
        )

        best_tasks_solution = [leaf2task[leaf_id] for leaf_id in best_leafs_solution]

        coalition_structure_2, allocation_structure_2, system_reward_2, iter_count_2, reassign_2  = _GreedyNE(
            original_allocation_structure=allocation_structure_1,
            selected_tasks=best_tasks_solution,
            selected_agents=agents_group,
        )

        total_iterations_count += iter_count_1 + iter_count_2
        total_reassignment_count += reassign_1 + reassign_2

        # Update allocation_solution
        allocation_solution = {}
        for i in allocation_structure_2:
            allocation_solution[i] = allocation_structure_2[i]

        if node_type == NodeType.AND:
            total_reward = 0
            for child_id in children_info[node_id]:
                child_tasks_descendants = [leaf2task[leaf_id] for leaf_id in leaves_list_info[child_id]]
                child_agents_group = sum([coalition_structure_2.get(task_id, []) for task_id in child_tasks_descendants], [])
                child_allocation_solution, child_system_reward, child_iter_count, child_reassign_count = aos_helper(child_id, child_agents_group, allocation_structure_2)
                # Update allocation_solution based on child_allocation_solution
                for j in child_allocation_solution:
                    allocation_solution[j] = child_allocation_solution[j]
                
                total_reward += child_system_reward

                total_iterations_count += child_iter_count
                total_reassignment_count += child_reassign_count
                
            return allocation_solution, total_reward, total_iterations_count, total_reassignment_count
        
        else: # if node_type == NodeType.OR:
            current_child_id = st_children_info[node_id][0]
            total_reward = system_reward_2
            final_allocation_solution = allocation_structure_2.copy()
            if not skip_initial_branch:
                child_allocation_solution, child_system_reward, child_iter_count, child_reassign_count = aos_helper(current_child_id, agents_group, allocation_structure_2)

                total_iterations_count += child_iter_count
                total_reassignment_count += child_reassign_count

                if child_system_reward > total_reward:
                    total_reward = child_system_reward
                    final_allocation_solution = child_allocation_solution

            for child_id in children_info[node_id]:
                if child_id == current_child_id:
                    continue
                # Bound pruning
                if use_branch_and_bound:
                    nodes_upper_bound_min = _get_upper_bound(child_id, agents_group)
                    reward_upper_bound = nodes_upper_bound_min[child_id]
                    if reward_upper_bound <= total_reward:
                        continue
                # Branch to child_id
                child_allocation_solution, child_system_reward, child_iter_count, child_reassign_count = aos_helper(child_id, agents_group, None)

                total_iterations_count += child_iter_count
                total_reassignment_count += child_reassign_count

                if child_system_reward > total_reward:
                    total_reward = child_system_reward
                    final_allocation_solution = child_allocation_solution

            # Update allocation_solution based on final_allocation_solution
            for j in final_allocation_solution:
                allocation_solution[j] = final_allocation_solution[j]

            return allocation_solution, total_reward, total_iterations_count, total_reassignment_count


    return aos_helper(root_node_id, list(range(0, len(agents))), allocation_structure_global)

In [23]:


def append_record(record, filename, typ):
    with open(filename, "a") as f:
        if typ != "":
            json.dump(record, f, default=typ)
        else:
            json.dump(record, f)
        f.write(os.linesep)
        f.close()

def gen_tasks_problem(task_num, agent_num, capNum, t_max_edge, a_min_edge):
    gamma = 1

    max_capVal = capNum
    max_capNum_task = capNum
    max_capNum_agent = capNum
    time_bound = 600

    capabilities = list(range(0, capNum))

    tasks = gen_tasks(task_num, max_capNum_task, capabilities)
    constraints = gen_constraints(agent_num, task_num, 1, a_min_edge, t_max_edge)
    a_taskInds = constraints[0]
    agents_cap, agents = gen_agents(a_taskInds, tasks, max_capNum_agent, capabilities, max_capVal)

    return capabilities, tasks, agents, constraints, gamma, t_max_edge, time_bound




def generate_tree_problem(tasks):
    """
    Encapsulate the generation of variables that represent the tree.

    This is so that only a number of data structures and variable types can be used as initial input to the MATA problem.

    All other variables must be generated within the solution function and/or must be counted as a procedure within the solution, and thus when calculating the solution speed, we must include the time taken to generate those other variables.
    """
    task_num = len(tasks)

    depth_info, parent_info, children_info, leaves_by_depth = gen_tree(task_num, min_leaf_depth=2)
    
    leaf_nodes = sum(leaves_by_depth, [])

    random.shuffle(leaf_nodes)

    leaf2task = {leaf_id : j for j, leaf_id in enumerate(leaf_nodes)}

    node_type_info = assign_node_type(depth_info, children_info, leaf_nodes)

    return node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info



def generate_tree_problem_breath_depth(breath, depth):
    """
    Encapsulate the generation of variables that represent the tree.

    This is so that only a number of data structures and variable types can be used as initial input to the MATA problem.

    All other variables must be generated within the solution function and/or must be counted as a procedure within the solution, and thus when calculating the solution speed, we must include the time taken to generate those other variables.
    """

    depth_info, parent_info, children_info, leaves_by_depth = gen_tree_simple(min_depth = depth, max_depth = depth, min_degree = breath, max_degree = breath, min_leaf_depth = depth)

    leaf_nodes = sum(leaves_by_depth, [])

    random.shuffle(leaf_nodes)

    leaf2task = {leaf_id : j for j, leaf_id in enumerate(leaf_nodes)}

    node_type_info = assign_node_type(depth_info, children_info, leaf_nodes)

    return node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info




def solve_get_upper_bound(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    start = time.perf_counter()
    leaves_list_info = get_leaves_list_info(parent_info=parent_info, leaf_nodes=leaf_nodes)
    nodes_constraints = get_nodes_constraints(node_type_info=node_type_info, leaves_list_info=leaves_list_info, leaf2task=leaf2task, constraints=constraints)
    tasks_capVecs = get_cap_vector_all(capabilities, tasks)
    ubcv_info = calculate_ubc_vectors(node_type_info, parent_info, leaves_list_info, leaf2task, tasks_capVecs, capabilities, query_nodeId=0)
    nodes_upper_bound = upperbound_node_all(children_info, ubcv_info, capabilities, agents, nodes_constraints, query_nodeId=0)
    nodes_upper_bound_min = upperbound_node_all_min(nodes_upper_bound, node_type_info, children_info, query_nodeId=0)
    end = time.perf_counter()
    
    print("UPPER BOUND:", nodes_upper_bound_min[0], "\ttime:", end - start)

    return nodes_upper_bound_min[0]


def solve_random_solution(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    start = time.perf_counter()
    rand_sol_alloc, rand_sol_reward = random_solution_and_or_tree(node_type_info, children_info, leaf2task, tasks, agents, constraints, gamma)
    end = time.perf_counter()

    print(f"Random: {rand_sol_reward}\ttime: {end - start}")

    return {
        "reward": rand_sol_reward,
        "time": end - start,
    }


def solve_treeGNE_1(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    start = time.perf_counter()
    result_c = treeGNE(
        node_type_info=node_type_info,
        children_info=children_info,
        parent_info=parent_info,
        task2leaf=leaf_nodes,
        leaf2task=leaf2task,
        tasks=tasks,
        agents=agents,
        constraints=constraints,
        gamma=gamma,
    )
    end = time.perf_counter()

    print(f"TreeGNE: {result_c[1][0]}\ttime: {end - start}\titeration: {result_c[2]}\tre-assignment {result_c[3]}")

    return {
        "reward": result_c[1][0],
        "time": end - start,
        "iteration": result_c[2],
        "re-assignment": result_c[3],
    }

def solve_treeGNE_2(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    
    start = time.perf_counter()
    result_c = treeGNE2(
        node_type_info=node_type_info,
        children_info=children_info,
        parent_info=parent_info,
        task2leaf=leaf_nodes,
        leaf2task=leaf2task,
        tasks=tasks,
        agents=agents,
        constraints=constraints,
        gamma=gamma,
    )
    end = time.perf_counter()

    print(f"TreeGNE2: {result_c[1][0]}\ttime: {end - start}\titeration: {result_c[2]}\tre-assignment {result_c[3]}")

    return {
        "reward": result_c[1][0],
        "time": end - start,
        "iteration": result_c[2],
        "re-assignment": result_c[3],
    }


def solve_fastTreeGNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    start = time.perf_counter()
    result_c = fastTreeGNE2(
        node_type_info=node_type_info,
        children_info=children_info,
        parent_info=parent_info,
        task2leaf=leaf_nodes,
        leaf2task=leaf2task,
        tasks=tasks,
        agents=agents,
        constraints=constraints,
        gamma=gamma,
    )
    end = time.perf_counter()

    print(f"fastTreeGNE2: {result_c[1][0]}\ttime: {end - start}\titeration: {result_c[2]}\tre-assignment {result_c[3]}")

    return {
        "reward": result_c[1][0],
        "time": end - start,
        "iteration": result_c[2],
        "re-assignment": result_c[3],
    }


def solve_simpleGNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    start = time.perf_counter()
    r_coalition_structure, r_sys_reward, r_iteration_count_1, r_re_assignment_count_1, r_iteration_count_2, r_re_assignment_count_2, r_loop_count = simpleGNE(
        node_type_info=node_type_info,
        children_info=children_info,
        leaf2task=leaf2task,
        agents=agents,
        tasks=tasks,
        constraints=constraints,
        gamma=gamma,
    )
    end = time.perf_counter()

    
    print(f"simpleGNE: {r_sys_reward}\ttime: {end - start}\titeration 1: {r_iteration_count_1}\tre-assignment 1 {r_re_assignment_count_1}\titeration 2: {r_iteration_count_2}\tre-assignment 2 {r_re_assignment_count_2}\tloop: {r_loop_count}")

    return {
        "reward": r_sys_reward,
        "time": end - start,
        "iteration_1": r_iteration_count_1,
        "re-assignment_1": r_re_assignment_count_1,
        "iteration_2": r_iteration_count_2,
        "re-assignment_2": r_re_assignment_count_2,
        "loop": r_loop_count,
    }


def solve_OrNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    
    start = time.perf_counter()
    leaves_list_info = get_leaves_list_info(parent_info=parent_info, leaf_nodes=leaf_nodes)
    nodes_constraints = get_nodes_constraints(node_type_info=node_type_info, leaves_list_info=leaves_list_info, leaf2task=leaf2task, constraints=constraints)
    tasks_capVecs = get_cap_vector_all(capabilities, tasks)
    ubcv_info = calculate_ubc_vectors(node_type_info, parent_info, leaves_list_info, leaf2task, tasks_capVecs, capabilities, query_nodeId=0)
    rorne_alloc, rorne_sys_reward, rorne_iteration_count, rorne_re_assignment_count = BnBOrNE(
        node_type_info=node_type_info,
        children_info=children_info,
        ubcv_info=ubcv_info,
        leaf2task=leaf2task,
        task2leaf=leaf_nodes,
        leaves_list_info=leaves_list_info,
        capabilities=capabilities,
        tasks=tasks,
        agents=agents,
        constraints=constraints,
        nodes_constraints=nodes_constraints,
        coalition_structure=None,
        gamma=gamma,
        root_node_id=0,
        use_branch_and_bound=False,
    )
    end = time.perf_counter()

    print(f"OrNE: {rorne_sys_reward}\ttime: {end - start}\titeration: {rorne_iteration_count}\tre-assignment {rorne_re_assignment_count}")

    return {
        "reward": rorne_sys_reward,
        "time": end - start,
        "iteration": rorne_iteration_count,
        "re-assignment": rorne_re_assignment_count,
    }


def solve_BnBOrNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    start = time.perf_counter()
    leaves_list_info = get_leaves_list_info(parent_info=parent_info, leaf_nodes=leaf_nodes)
    nodes_constraints = get_nodes_constraints(node_type_info=node_type_info, leaves_list_info=leaves_list_info, leaf2task=leaf2task, constraints=constraints)
    tasks_capVecs = get_cap_vector_all(capabilities, tasks)
    ubcv_info = calculate_ubc_vectors(node_type_info, parent_info, leaves_list_info, leaf2task, tasks_capVecs, capabilities, query_nodeId=0)
    rorne_alloc, rorne_sys_reward, rorne_iteration_count, rorne_re_assignment_count = BnBOrNE(
        node_type_info=node_type_info,
        children_info=children_info,
        ubcv_info=ubcv_info,
        leaf2task=leaf2task,
        task2leaf=leaf_nodes,
        leaves_list_info=leaves_list_info,
        capabilities=capabilities,
        tasks=tasks,
        agents=agents,
        constraints=constraints,
        nodes_constraints=nodes_constraints,
        coalition_structure=None,
        gamma=gamma,
        root_node_id=0,
    )
    end = time.perf_counter()

    print(f"BnBOrNE: {rorne_sys_reward}\ttime: {end - start}\titeration: {rorne_iteration_count}\tre-assignment {rorne_re_assignment_count}")

    return {
        "reward": rorne_sys_reward,
        "time": end - start,
        "iteration": rorne_iteration_count,
        "re-assignment": rorne_re_assignment_count,
    }


def solve_BnBOrNE_skip(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes):
    
    start = time.perf_counter()
    leaves_list_info = get_leaves_list_info(parent_info=parent_info, leaf_nodes=leaf_nodes)
    nodes_constraints = get_nodes_constraints(node_type_info=node_type_info, leaves_list_info=leaves_list_info, leaf2task=leaf2task, constraints=constraints)
    tasks_capVecs = get_cap_vector_all(capabilities, tasks)
    ubcv_info = calculate_ubc_vectors(node_type_info, parent_info, leaves_list_info, leaf2task, tasks_capVecs, capabilities, query_nodeId=0)
    rorne_alloc, rorne_sys_reward, rorne_iteration_count, rorne_re_assignment_count = BnBOrNE(
        node_type_info=node_type_info,
        children_info=children_info,
        ubcv_info=ubcv_info,
        leaf2task=leaf2task,
        task2leaf=leaf_nodes,
        leaves_list_info=leaves_list_info,
        capabilities=capabilities,
        tasks=tasks,
        agents=agents,
        constraints=constraints,
        nodes_constraints=nodes_constraints,
        coalition_structure=None,
        gamma=gamma,
        root_node_id=0,
        skip_initial_branch=True,
    )
    end = time.perf_counter()

    print(f"BnBOrNEskip: {rorne_sys_reward}\ttime: {end - start}\titeration: {rorne_iteration_count}\tre-assignment {rorne_re_assignment_count}")

    return {
        "reward": rorne_sys_reward,
        "time": end - start,
        "iteration": rorne_iteration_count,
        "re-assignment": rorne_re_assignment_count,
    }


def solve_tree(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info):

    result_row = {}

    task_num = len(tasks)

    result_row["task_num"] = task_num

    max_depth = len(leaves_by_depth) - 1
    min_depth = 0
    while len(leaves_by_depth[min_depth]) == 0:
        min_depth += 1

    branching_factor = (len(node_type_info) - 1) / len(children_info)
    min_degree = min([len(c) for c in children_info.values()])
    max_degree = max([len(c) for c in children_info.values()])
    num_internal_nodes = len(children_info)

    result_row["tree_info"] = {
        "max_depth": max_depth,
        "min_depth": min_depth,
        "branching_factor": branching_factor,
        "min_degree": min_degree,
        "max_degree": max_degree,
        "num_internal_nodes": num_internal_nodes,
    }

    result_row["upper_bound"] = solve_get_upper_bound(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    result_row["random_solution"] = solve_random_solution(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    result_row["treeGNE"] = solve_treeGNE_1(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    # result_row["treeGNE2"] = solve_treeGNE_2(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    result_row["fastTreeGNE2"] = solve_fastTreeGNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    result_row["simpleGNE"] = solve_simpleGNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    # result_row["AOsearchGNE"] = solve_AOSearchGNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    # result_row["OrNE"] = solve_OrNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    result_row["BnBOrNE"] = solve_BnBOrNE(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    # result_row["BnBOrNEskip"] = solve_BnBOrNE_skip(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes)

    return result_row


# def main_tree(capabilities, tasks, agents, constraints, gamma):

#     """
#     Driver code for algorithms related to AND-OR goal tree.
#     """
#     node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info = generate_tree_problem(tasks)

#     result_row = solve_tree(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info)

#     return result_row



def main_run(task_num, agent_num, capNum, t_max_edge, a_min_edge, ex_identifier = None, save_to_file = None, breath = None, depth = None):
    
    gamma = 1

    # max_capVal = capNum
    # max_capNum_task = capNum
    # max_capNum_agent = capNum
    # time_bound = 600

    # capabilities = list(range(0, capNum))

    # # agent_num = np.random.randint(task_num,3*task_num)
    # tasks = gen_tasks(task_num, max_capNum_task, capabilities)
    # constraints = gen_constraints(agent_num, task_num, 1, a_min_edge, t_max_edge)
    # a_taskInds = constraints[0]
    # agents_cap, agents = gen_agents(a_taskInds, tasks, max_capNum_agent, capabilities, max_capVal)
    # # num_com = np.prod([1 if a_taskInds[i] == [] else len(a_taskInds[i])+1 for i in range(0,agent_num)])

    if breath is not None and depth is not None:
        node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info = generate_tree_problem_breath_depth(breath, depth)

        task_num = len(leaf_nodes)

        capabilities, tasks, agents, constraints, gamma, t_max_edge, time_bound = gen_tasks_problem(task_num, agent_num, capNum, t_max_edge, a_min_edge)
    
    else:
        capabilities, tasks, agents, constraints, gamma, t_max_edge, time_bound = gen_tasks_problem(task_num, agent_num, capNum, t_max_edge, a_min_edge)

        node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info = generate_tree_problem(tasks)

    a_taskInds = constraints[0]

    num_com = reduce(
        mul,
        [
            1 if a_taskInds[i] == [] else len(a_taskInds[i]) + 1
            for i in range(0, agent_num)
        ],
    )

    up = upperBound(capabilities, tasks, agents)

    up2 = upperBound_ver2(capabilities, tasks, agents, constraints)
    print("UP:", up, "  UP2:", up2)

    result_info = {
        "ex_identifier": ex_identifier,
        "task_num": task_num,
        "agent_num": agent_num,
        "capNum": capNum,
        "up": up,
        "up2": up2,
    }
    #         data = {"ex_identifier":ex_identifier,"tasks":tasks,"constraints":constraints,"agents_cap":agents_cap,"agents":agents}

    a_den = [len(c) for c in constraints[0]]
    t_den = [len(c) for c in constraints[1]]
    result_info["a_den_avg"] = statistics.mean(a_den)
    result_info["a_den_max"] = max(a_den)
    result_info["a_den_min"] = min(a_den)

    result_info["t_den_avg"] = statistics.mean(t_den)
    result_info["t_den_max"] = max(t_den)
    result_info["t_den_min"] = min(t_den)
    result_info["t_max_edge"] = t_max_edge

    print(
        "density", t_max_edge, "task_num:", task_num, "  agent_num:", agent_num
    )
    print(
        "a_den_avg",
        result_info["a_den_avg"],
        "a_den_max:",
        result_info["a_den_max"],
        "  a_den_min:",
        result_info["a_den_min"],
    )

    print(
        "t_den_avg",
        result_info["t_den_avg"],
        "t_den_max:",
        result_info["t_den_max"],
        "  t_den_min:",
        result_info["t_den_min"],
    )
    print("-----------------------------------")
    print("Heterogeneous Tasks")
    print("-----------------------------------")

    start = time.perf_counter()
    r = eGreedy2(agents, tasks, constraints, gamma=gamma)
    end = time.perf_counter()
    result_info["g"] = r[1]
    result_info["g_iter"] = r[2]
    result_info["g_reass"] = r[3]
    result_info["g_t"] = end - start
    print(
        "eGreedy:",
        "\ttime:",
        result_info["g_t"],
        "\tresult:",
        result_info["g"],
        "\titeration:",
        result_info["g_iter"],
        "\tre-assignment",
        result_info["g_reass"],
    )

    start = time.perf_counter()
    rand_sol_a, rand_sol_reward = random_solution_heterogeneous(agents, tasks, constraints, gamma=gamma)
    end = time.perf_counter()
    print("Random Solution:", "\ttime:", end - start, "\tresult:", rand_sol_reward)

    print("-----------------------------------")
    print("AND-OR Tree Tasks")
    print("-----------------------------------")

    result_row = solve_tree(capabilities, tasks, agents, constraints, gamma, node_type_info, parent_info, children_info, leaf2task, leaf_nodes, leaves_by_depth, depth_info)
    result_row["info"] = result_info

    if save_to_file:
        append_record(result_row, save_to_file, typ="")

    return result_row


def main_run_1(args):
    return main_run(*args)


def main_single(filename = "local-results.jsonl", remove_file = False):

    if remove_file:
        if os.path.exists(filename):
            os.remove(filename)
    
    ex_identifier = 0

    for task_num in range(100, 1100, 100):
        for agent_tasks_ratio in range(2, 5):
            agent_num = task_num * agent_tasks_ratio
            for capNum in range(10, 15):
                a_min_edge = 2
                min_t_max_edge = max(math.ceil((agent_num * a_min_edge) / task_num), 10)
                max_t_max_edge = min_t_max_edge + 5 * 3
                for t_max_edge in range(min_t_max_edge, max_t_max_edge + 1, 3):
                    run_num = 3
                    for run in range(0, run_num):
                        # print("----------------------------------------------------------------------")
                        # print("EXPERIMENT")
                        # print("----------------------------------------------------------------------")
                        # result_row = main_run(task_num, agent_num, capNum, t_max_edge, a_min_edge)
                        # # append data and result
                        # files = {"local-results.jsonl": [result_row, ""]}

                        # for filename in list(files.keys()):
                        #     append_record(files[filename][0], filename, typ=files[filename][1])
                        ex_identifier += 1
                        print("----------------------------------------------------------------------")
                        print("EX IDENTIFIER:", ex_identifier)
                        print("----------------------------------------------------------------------")
                        result_row = main_run(task_num, agent_num, capNum, t_max_edge, a_min_edge, ex_identifier, filename)


def main_single_breath_depth(filename = "local-results.jsonl", remove_file = False):

    if remove_file:
        if os.path.exists(filename):
            os.remove(filename)
    
    ex_identifier = 0

    min_task_num = 100
    max_task_num = 1000
    breath_depth_task_nums = [(breath, depth, breath ** depth) for breath in range(2, 10) for depth in range(2, 10) if (breath ** depth >= min_task_num) and (breath ** depth < max_task_num)]

    breath_depth_task_nums = sorted(breath_depth_task_nums, key=lambda x: x[2])

    for breath, depth, task_num in breath_depth_task_nums:
        for agent_tasks_ratio in range(2, 5):
            agent_num = task_num * agent_tasks_ratio
            for capNum in range(10, 15):
                a_min_edge = 2
                min_t_max_edge = max(math.ceil((agent_num * a_min_edge) / task_num), 10)
                max_t_max_edge = min_t_max_edge + 5 * 3
                for t_max_edge in range(min_t_max_edge, max_t_max_edge + 1, 3):
                    run_num = 3
                    for run in range(0, run_num):
                        # print("----------------------------------------------------------------------")
                        # print("EXPERIMENT")
                        # print("----------------------------------------------------------------------")
                        # result_row = main_run(task_num, agent_num, capNum, t_max_edge, a_min_edge)
                        # # append data and result
                        # files = {"local-results.jsonl": [result_row, ""]}

                        # for filename in list(files.keys()):
                        #     append_record(files[filename][0], filename, typ=files[filename][1])
                        ex_identifier += 1
                        print("----------------------------------------------------------------------")
                        print("EX IDENTIFIER:", ex_identifier)
                        print("Breath:", breath, "Depth:", depth)
                        print("----------------------------------------------------------------------")
                        result_row = main_run(task_num, agent_num, capNum, t_max_edge, a_min_edge, ex_identifier, filename, breath=breath, depth=depth)




def main_cli_full_args():

    parser = argparse.ArgumentParser(description="Run the main function.")
    parser.add_argument(
        "--task_num",
        type=int,
        default=100,
        help="Number of tasks to generate.",
    )
    parser.add_argument(
        "--agent_num",
        type=int,
        default=200,
        help="Number of agents to generate.",
    )
    parser.add_argument(
        "--capNum",
        type=int,
        default=10,
        help="Number of capabilities to generate.",
    )
    parser.add_argument(
        "--t_max_edge",
        type=int,
        default=15,
        help="Maximum number of edges to generate.",
    )
    parser.add_argument(
        "--a_min_edge",
        type=int,
        default=2,
        help="Minimum number of edges to generate.",
    )
    parser.add_argument(
        "--iterations",
        type=int,
        default=0,
        help="Number of iterations to run.",
    )

    args = parser.parse_args()

    if args.iterations == 0:
        args.iterations = 3

    for run in range(0, args.iterations):
        result_row = main_run(
            args.task_num,
            args.agent_num,
            args.capNum,
            args.t_max_edge,
            args.a_min_edge,
            None,
            'results-cli.jsonl',
        )


def main_cli():

    parser = argparse.ArgumentParser(description="Run by task_num.")
    parser.add_argument(
        "--task_num",
        type=int,
        default=100,
        help="Number of tasks to generate.",
    )
    parser.add_argument(
        "--iterations",
        type=int,
        default=0,
        help="Number of iterations to run.",
    )

    args = parser.parse_args()

    if args.iterations == 0:
        args.iterations = 3

    task_num = args.task_num

    for agent_tasks_ratio in range(2, 5):
        agent_num = task_num * agent_tasks_ratio
        for capNum in range(10, 15):
            a_min_edge = 2
            min_t_max_edge = max(math.ceil((agent_num * a_min_edge) / task_num), 10)
            max_t_max_edge = max(min_t_max_edge, 50)
            for t_max_edge in range(min_t_max_edge, max_t_max_edge + 1):
                run_num = 3
                for run in range(0, run_num):
                    result_row = main_run(task_num, agent_num, capNum, t_max_edge, a_min_edge, None, "local-results.jsonl")




def main_multiprocessing():

    args = []

    ex_identifier = 0

    for task_num in range(300, 1100, 100):
        for agent_tasks_ratio in range(2, 5):
            agent_num = task_num * agent_tasks_ratio
            for capNum in range(10, 15):
                a_min_edge = 2
                min_t_max_edge = max(math.ceil((agent_num * a_min_edge) / task_num), 10)
                max_t_max_edge = min_t_max_edge + 5 * 3
                for t_max_edge in range(min_t_max_edge, max_t_max_edge + 1, 5):
                    run_num = 3
                    for run in range(0, run_num):
                        # print("----------------------------------------------------------------------")
                        # print("EXPERIMENT")
                        # print("----------------------------------------------------------------------")
                        # result_row = main_run(task_num, agent_num, capNum, t_max_edge, a_min_edge)
                        # # append data and result
                        # files = {"local-results.jsonl": [result_row, ""]}

                        # for filename in list(files.keys()):
                        #     append_record(files[filename][0], filename, typ=files[filename][1])
                        ex_identifier += 1
                        args.append((task_num, agent_num, capNum, t_max_edge, a_min_edge, ex_identifier, "results-multiprocessing.jsonl"))

    with Pool(5) as p:
        p.map(main_run_1, args) 

def main_multithread():

    ex_identifier = 0

    args = []

    for task_num in range(100, 1000, 100):
        for agent_tasks_ratio in range(2, 5):
            agent_num = task_num * agent_tasks_ratio
            for capNum in range(10, 15):
                a_min_edge = 2
                min_t_max_edge = max(math.ceil((agent_num * a_min_edge) / task_num), 10)
                max_t_max_edge = max(min_t_max_edge, 50)
                for t_max_edge in range(min_t_max_edge, max_t_max_edge + 1):
                    run_num = 3
                    for run in range(0, run_num):
                        ex_identifier += 1
                        args.append((task_num, agent_num, capNum, t_max_edge, a_min_edge, ex_identifier, "results-multithread.jsonl"))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(main_run_1, args)


In [24]:

if __name__ == "__main__":
    main_single_breath_depth(filename='local-results.jsonl')
    # main_single(filename='results-1000-3.jsonl')
    # main_multiprocessing()

----------------------------------------------------------------------
EX IDENTIFIER: 1
Breath: 5 Depth: 3
----------------------------------------------------------------------
UP: 6155   UP2: 6140
density 10 task_num: 125   agent_num: 250
a_den_avg 4.996 a_den_max: 10   a_den_min: 1
t_den_avg 9.992 t_den_max: 10   t_den_min: 9
-----------------------------------
Heterogeneous Tasks
-----------------------------------
eGreedy: 	time: 0.5589846999500878 	result: 5310 	iteration: 275 	re-assignment 30
Random Solution: 	time: 0.006035399972461164 	result: 3268
-----------------------------------
AND-OR Tree Tasks
-----------------------------------
UPPER BOUND: 1616 	time: 0.010438300028908998
Random: 816	time: 0.004614999983459711
TreeGNE: 1507	time: 0.34110610000789165	iteration: 97	re-assignment 3
fastTreeGNE2: 1507	time: 0.5012410000199452	iteration: 256	re-assignment 12
simpleGNE: 1511	time: 0.27074939996236935	iteration 1: 483	re-assignment 1 95	iteration 2: 102	re-assignment 2 7	l